# Counter-Samples: WACV Paper

## Notebook Overview
This Jupyter Notebook is designed to serve as an interactive platform for running our baselines experiment. The core aim of this notebook is to offer reviewers the abillity to run our experiment while configuring the hyper parameters as they like.

Thsi notebook has been written to evalaute one blackbox attack (Bandit) and can be changed manually to other attacks as needed (see below in the *Hyperparameters Configuration* section).  Due to the limited resources in COLAB, the notbook will only run the attack on 100 samples from CIFAR-10 dataset. The code expect a colab instace connected to a cuda GPU. Please be sure to change your runtime accordingly.

## Organization
- Setup
- Hyperparameter Configuration
- Results and Analysis


In the dedicated section titled "Hyperparameter Configuration", reviewers are granted the flexibility to define and adjust the hyperparameters relevant to both the attacks and our defence method.

Upon the execution of the experiments, all outcomes will be systematically saved in a newly created directory named "Results". This directory is structured to store the results in an organized manner, enabling easy access and analysis.

Through this notebook, we aim to provide a transparent and replicable evaluation framework that not only demonstrates the capabilities of our proposed defense but also encourages rigorous scrutiny and validation by the ECCV community.

**Note, you must have a GPU present.**

# Installing packages and getting the data

In [ ]:
!pip install advertorch
!pip install adversarial-robustness-toolbox

In [ ]:
!git clone https://github.com/SCLBD/BlackboxBench.git

In [ ]:
%cd /content/BlackboxBench


In [ ]:
!git checkout 320e02cca886a0777851b8061b01b1c76eb67e7e

In [ ]:
!mv "/content/BlackboxBench/attacks/decision-based attacks" "/content/BlackboxBench/attacks/decision"
!mv "/content/BlackboxBench/attacks/score-based attacks" "/content/BlackboxBench/attacks/score"


In [ ]:
%cd ..

/content


In [ ]:
!pip install -r '/content/BlackboxBench/requirements.txt'


We are downloding the CIFAR10 dataset from : https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz <br> into 'BlackBoxBench/data' folder we create


In [ ]:
!mkdir '/content/BlackboxBench/data'

In [ ]:
!wget -P /content/BlackboxBench/data https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz


In [ ]:
!tar -xzvf /content/BlackboxBench/data/cifar-10-python.tar.gz -C /content/BlackboxBench/data


# Adding our code for running the experiment

In [ ]:
%%writefile /content/BlackboxBench/attacks/score/score_black_box_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
from torch import Tensor as t

from utils.compute import l2_proj_maker, linf_proj_maker
import sys

class ScoreBlackBoxAttack(object):
    def __init__(self, max_loss_queries=np.inf,
                 max_extra_queries=np.inf,
                 epsilon=0.5, p='inf', lb=0., ub=1.,batch_size = 50, name = "nes"):
        """
        :param max_loss_queries: max number of calls to model per data point
        :param max_extra_queries: max number of calls to early stopping extraerion per data point
        :param epsilon: perturbation limit according to lp-ball
        :param p: norm for the lp-ball constraint
        :param lb: minimum value data point can take in any coordinate
        :param ub: maximum value data point can take in any coordinate
        """
        assert p in ['inf', '2'], "L-{} is not supported".format(p)

        self.epsilon = epsilon
        self.p = p
        self.batch_size = batch_size
        self.max_loss_queries = max_loss_queries
        self.max_extra_queries = max_extra_queries
        self.list_loss_queries = torch.zeros(1, self.batch_size)
        self.total_loss_queries = 0
        self.total_extra_queries = 0
        self.total_successes = 0
        self.total_failures = 0
        self.lb = lb
        self.ub = ub
        self.name = name
        # the _proj method takes pts and project them into the constraint set:
        # which are
        #  1. epsilon lp-ball around xs
        #  2. valid data pt range [lb, ub]
        # it is meant to be used within `self.run` and `self._perturb`
        self._proj = None
        # a handy flag for _perturb method to denote whether the provided xs is a
        # new batch (i.e. the first iteration within `self.run`)
        self.is_new_batch = False

    def result(self):
        """
        returns a summary of the attack results (to be tabulated)
        :return:
        """
        list_loss_queries = self.list_loss_queries[1:].view(-1)
        mask = list_loss_queries > 0
        list_loss_queries = list_loss_queries[mask]
        self.total_loss_queries = int(self.total_loss_queries)
        self.total_extra_queries = int(self.total_extra_queries)
        self.total_successes = int(self.total_successes)
        self.total_failures = int(self.total_failures)
        return {
            "total_loss_queries": self.total_loss_queries,
            "total_extra_queries": self.total_extra_queries,
            "average_num_loss_queries": "NaN" if self.total_successes == 0 else self.total_loss_queries / self.total_successes,
            "average_num_extra_queries": "NaN" if self.total_successes == 0 else self.total_extra_queries / self.total_successes,
            "median_num_loss_queries": "NaN" if self.total_successes == 0 else torch.median(list_loss_queries).item(),
            "total_queries": self.total_extra_queries + self.total_loss_queries,
            "average_num_queries": "NaN" if self.total_successes == 0 else (self.total_extra_queries + self.total_loss_queries) / self.total_successes,
            "total_successes": self.total_successes,
            "total_failures": self.total_failures,
            "failure_rate": "NaN" if self.total_successes + self.total_failures == 0 else self.total_failures / (self.total_successes + self.total_failures),
            "config": self._config()
        }

    def _config(self):
        """
        return the attack's parameter configurations as a dict
        :return:
        """
        raise NotImplementedError

    def _perturb(self, xs_t, loss_fct):
        """
        :param xs_t: batch_size x dim x .. (torch tensor)
        :param loss_fct: function to query (the attacker would like to maximize) (batch_size data pts -> R^{batch_size}
        :return: suggested xs as a (torch tensor)and the used number of queries per data point
            i.e. a tuple of (batch_size x dim x .. tensor, batch_size array of number queries used)
        """
        raise NotImplementedError

    def proj_replace(self, xs_t, sugg_xs_t, dones_mask_t):
        sugg_xs_t = self._proj(sugg_xs_t)
        # replace xs only if not done
        xs_t = sugg_xs_t * (1. - dones_mask_t) + xs_t * dones_mask_t
        return xs_t

    def run(self, xs, loss_fct, early_stop_extra_fct):
        """
        attack with `xs` as data points using the oracle `l` and
        the early stopping extraerion `early_stop_extra_fct`
        :param xs: data points to be perturbed adversarially (numpy array)
        :param loss_fct: loss function (m data pts -> R^m)
        :param early_stop_extra_fct: early stop function (m data pts -> {0,1}^m)
                ith entry is 1 if the ith data point is misclassified
        :return: a dict of logs whose length is the number of iterations
        """
        # convert to tensor
        xs_t = t(xs)

        batch_size = xs.shape[0]
        num_axes = len(xs.shape[1:])
        num_loss_queries = torch.zeros(batch_size)
        num_extra_queries = torch.zeros(batch_size)

        dones_mask = early_stop_extra_fct(xs_t)
        correct_classified_mask = ~dones_mask

        # list of logs to be returned
        logs_dict = {
            'total_loss': [],
            'total_successes': [],
            'total_failures': [],
            'iteration': [],
            'total_loss_queries': [],
            'total_extra_queries': [],
            'total_queries': [],
            'num_loss_queries_per_iteration': [],
            'num_extra_queries_per_iteration': []
        }

        # init losses for performance tracking
        losses = torch.zeros(batch_size)

        # make a projector into xs lp-ball and within valid pixel range
        if self.p == '2':
            _proj = l2_proj_maker(xs_t, self.epsilon)
            self._proj = lambda _: torch.clamp(_proj(_), self.lb, self.ub)
        elif self.p == 'inf':
            _proj = linf_proj_maker(xs_t, self.epsilon)
            self._proj = lambda _: torch.clamp(_proj(_), self.lb, self.ub)
        else:
            raise Exception('Undefined l-p!')

        # iterate till model evasion or budget exhaustion
        self.is_new_batch = True
        its = 0
        while True:
            if torch.any(num_loss_queries +num_extra_queries >= self.max_loss_queries ):
                print('Number of queries limit reached.')
                break
            if torch.any(num_loss_queries >= self.max_loss_queries):
                print("#loss queries exceeded budget, exiting")
                break
            if torch.any(num_extra_queries >= self.max_extra_queries):
                print("#extra_queries exceeded budget, exiting")
                break
            if torch.all(dones_mask):
                print("all data pts are misclassified, exiting")
                break
            # propose new perturbations
            sugg_xs_t, num_loss_queries_per_step = self._perturb(xs_t, loss_fct)

            # project around xs and within pixel range and
            # replace xs only if not done
            ##updated x here
            xs_t = self.proj_replace(xs_t, sugg_xs_t, (dones_mask.reshape(-1, *[1] * num_axes).float()))

            # update number of queries (note this is done before updating dones_mask)
            num_loss_queries += num_loss_queries_per_step * (~dones_mask)
            num_extra_queries += (~dones_mask)
            losses = loss_fct(xs_t) * (~dones_mask) + losses * dones_mask

            # update dones mask
            dones_mask = dones_mask | early_stop_extra_fct(xs_t)
            success_mask = dones_mask * correct_classified_mask
            its += 1

            self.is_new_batch = False


            if its % 500 == 0:
                success_mask = dones_mask * correct_classified_mask
                total_successes = float(success_mask.sum())
                print ("Iteration : ", its, 'ave_loss_queries : ', ((num_loss_queries * success_mask).sum() / total_successes).item(),\
                    "ave_extra_queries : ", ((num_extra_queries * success_mask).sum()  / total_successes).item(), \
                    "ave_queries : ", ((num_loss_queries * success_mask).sum() / total_successes + (num_extra_queries * success_mask).sum()  / total_successes).item(), \
                    "successes : ", success_mask.sum().item() / float(success_mask.shape[0]), \
                    "failures : ", ((~dones_mask) * correct_classified_mask).sum().item()  / float(success_mask.shape[0]))
                sys.stdout.flush()



        success_mask = dones_mask * correct_classified_mask
        self.total_loss_queries += (num_loss_queries * success_mask).sum()
        self.total_extra_queries += (num_extra_queries * success_mask).sum()
        self.list_loss_queries = torch.cat([self.list_loss_queries, torch.zeros(1, batch_size)], dim=0)
        self.list_loss_queries[-1] = num_loss_queries * success_mask
        self.total_successes += success_mask.sum()
        self.total_failures += ((~dones_mask) * correct_classified_mask).sum()


        # set self._proj to None to ensure it is intended use
        self._proj = None

        return logs_dict
'''

MIT License
Copyright (c) 2019 Abdullah Al-Dujaili
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/score/score_black_box_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/bandit_attack.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch

from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step, step, eg_step, upsample_maker, l2_step


class BanditAttack(ScoreBlackBoxAttack):
    """
    Bandit Attack
    """

    def __init__(self,
                 max_loss_queries,
                 epsilon, p,
                 fd_eta, lr,
                 prior_exploration, prior_size, data_size, prior_lr,
                 lb, ub, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param fd_eta: forward difference step
        :param lr: learning rate of NES step
        :param prior_exploration: exploration noise
        :param prior_size: prior height/width (this is applicable only to images), you can disable it by setting it to
            None (it is assumed to prior_size = prior_height == prior_width)
        :param data_size: data height/width (applicable to images of the from `c x h x w`, you can ignore it
            by setting it to none, it is assumed that data_size = height = width
        :param prior_lr: learning rate in the prior space
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size= batch_size,
                         name = "Bandit")
        # other algorithmic parameters
        self.fd_eta = fd_eta
        # learning rate
        self.lr = lr
        # data size
        self.data_size = data_size

        # prior setup:
        # 1. step function
        if self.p == '2':
            self.prior_step = step
        elif self.p == 'inf':
            self.prior_step = eg_step
        else:
            raise Exception("Invalid p for l-p constraint")
        # 2. prior placeholder
        self.prior = None
        # prior size
        self.prior_size = prior_size
        # prior exploration
        self.prior_exploration = prior_exploration
        # 3. prior upsampler
        self.prior_upsample_fct = None if self.prior_size is None else upsample_maker(data_size, data_size)
        self.prior_lr = prior_lr

    def _perturb(self, xs_t, loss_fct):
        """
        The core of the bandit algorithm
        since this is compute intensive, it is implemented with torch support to push ops into gpu (if available)
        however, the input / output are numpys
        :param xs: numpy
        :return new_xs: returns a torch tensor
        """
        if xs_t.dim() != 2 and self.prior_size is not None:  # for cifar10 and imagenet data needs to be transpose into c x  h x w for upsample method
            xs_t = xs_t.transpose(1, 3)
        _shape = list(xs_t.shape)
        eff_shape = list(xs_t.shape)
        # since the upsampling assumes xs_t is batch_size x c x h x w. This is not the case for mnist,
        # which is batch_size x dim, let's take care of that below
        if len(_shape) == 2:
            eff_shape = [_shape[0], 1, self.data_size, self.data_size]
        if self.prior_size is None:
            prior_shape = eff_shape
        else:
            prior_shape = eff_shape[:-2] + [self.prior_size] * 2
        # reset the prior if xs  is a new batch
        if self.is_new_batch:
            self.prior = torch.zeros(prior_shape)
        # create noise for exploration, estimate the gradient, and take a PGD step
        # exp_noise = torch.randn(prior_shape) / (np.prod(prior_shape[1:]) ** 0.5)  # according to the paper
        exp_noise = torch.randn(prior_shape)
        # Query deltas for finite difference estimator
        if self.prior_size is None:
            q1 = step(self.prior, exp_noise, self.prior_exploration)
            q2 = step(self.prior, exp_noise, - self.prior_exploration)
        else:
            q1 = self.prior_upsample_fct(step(self.prior, exp_noise, self.prior_exploration))
            q2 = self.prior_upsample_fct(step(self.prior, exp_noise, - self.prior_exploration))
        # Loss points for finite difference estimator
        if xs_t.dim() != 2 and self.prior_size is not None:
            l1 = loss_fct(l2_step(xs_t, q1.view(_shape), self.fd_eta).transpose(1, 3))
            l2 = loss_fct(l2_step(xs_t, q2.view(_shape), self.fd_eta).transpose(1, 3))
        else:
            l1 = loss_fct(l2_step(xs_t, q1.view(_shape), self.fd_eta))
            l2 = loss_fct(l2_step(xs_t, q2.view(_shape), self.fd_eta))
        # finite differences estimate of directional derivative
        est_deriv = (l1 - l2) / (self.fd_eta * self.prior_exploration)
        # 2-query gradient estimate
        # Note: Ilyas' implementation multiply the below by self.prior_exploration (different from pseudocode)
        # This should not affect the result as the `self.prior_lr` can be adjusted accordingly
        est_grad = torch.Tensor(est_deriv.reshape(-1, *[1] * len(prior_shape[1:]))) * exp_noise
        # update prior with the estimated gradient:
        self.prior = self.prior_step(self.prior, est_grad, self.prior_lr)
        # gradient step in the data space
        if self.prior_size is None:
            gs = self.prior.clone()
        else:
            gs = self.prior_upsample_fct(self.prior)
        if xs_t.dim() != 2 and self.prior_size is not None:
            gs = gs.transpose(1, 3)
            xs_t = xs_t.transpose(1, 3)
            _shape = list(xs_t.shape)
        # perform the step
        new_xs = lp_step(xs_t, gs.view(_shape), self.lr, self.p)
        return new_xs, 2 * torch.ones(_shape[0])

    def _config(self):
        return {
            "name": self.name,
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "lr": self.lr,
            "prior_lr": self.prior_lr,
            "prior_exploration": self.prior_exploration,
            "prior_size": self.prior_size,
            "data_size": self.data_size,
            "fd_eta": self.fd_eta,
            "attack_name": self.__class__.__name__
        }

'''

MIT License

Copyright (c) 2019 Abdullah Al-Dujaili

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/score/bandit_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/nes_attack.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
from torch import Tensor as t

from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step


class NESAttack(ScoreBlackBoxAttack):
    """
    NES Attack
    """

    def __init__(self, max_loss_queries, epsilon, p, fd_eta, lr, q, lb, ub, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param fd_eta: forward difference step
        :param lr: learning rate of NES step
        :param q: number of noise samples per NES step
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size= batch_size,
                         name = "NES")
        self.q = q
        self.fd_eta = fd_eta
        self.lr = lr

    def _perturb(self, xs_t, loss_fct):
        _shape = list(xs_t.shape)
        dim = np.prod(_shape[1:])
        num_axes = len(_shape[1:])
        gs_t = torch.zeros_like(xs_t)
        for _ in range(self.q):
            # exp_noise = torch.randn_like(xs_t) / (dim ** 0.5)
            exp_noise = torch.randn_like(xs_t)
            fxs_t = xs_t + self.fd_eta * exp_noise
            bxs_t = xs_t - self.fd_eta * exp_noise
            est_deriv = (loss_fct(fxs_t) - loss_fct(bxs_t)) / (4. * self.fd_eta)
            gs_t += t(est_deriv.reshape(-1, *[1] * num_axes)) * exp_noise
        # perform the step
        new_xs = lp_step(xs_t, gs_t, self.lr, self.p)
        return new_xs, 2 * self.q * torch.ones(_shape[0])

    def _config(self):
        return {
            "name": self.name,
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "lr": self.lr,
            "q": self.q,
            "fd_eta": self.fd_eta,
            "attack_name": self.__class__.__name__
        }

'''

Original License

MIT License

Copyright (c) 2019 Abdullah Al-Dujaili

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/score/nes_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/parsimonious_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from torch import Tensor as t
import heapq
import math
import itertools
import torch

from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step


class ParsimoniousAttack(ScoreBlackBoxAttack):
    """
    Parsimonious Attack
    """

    def __init__(self, max_loss_queries, epsilon, p, block_size, block_batch_size, EOT, lb, ub, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size=batch_size,
                         name='ECO')
        self.block_sizeo = block_size
        self.batch_sizeo = block_batch_size
        self.no_hier = False
        self.max_iters = 1
        self.EOT = EOT

    def _split_block(self, upper_left, lower_right, block_size):
        """
        Split an image into a set of blocks.
        Note that a block consists of [upper_left, lower_right, channel]

        Args:
          upper_left: [x, y], the coordinate of the upper left of an image
          lower_right: [x, y], the coordinate of the lower right of an image
          block_size: int, the size of a block

        Return:
          blocks: list, a set of blocks
        """
        blocks = []
        xs = torch.arange(upper_left[0], lower_right[0], block_size)
        ys = torch.arange(upper_left[1], lower_right[1], block_size)
        for x, y in itertools.product(xs, ys):
            for c in range(3):
                blocks.append([[x, y], [x + block_size, y + block_size], c])
        return blocks

    def _perturb_image(self, image, noise):
        adv_image = image + noise
        adv_image = torch.clamp(adv_image, 0, self.ub)
        return adv_image


    def _flip_noise(self, noise, block):
        """Filp the sign of perturbation on a block.
            Args:
              noise: numpy array of size [1, 3, 32, 32], a noise
              block: [upper_left, lower_right, channel], a block

            Returns:
              noise_new: numpy array with size [1,3, 32, 32], an updated noise
        """
        noise_new = noise.clone()
        upper_left, lower_right, channel = block
        noise_new[0, upper_left[0]:lower_right[0], upper_left[1]:lower_right[1],channel] *= -1
        return noise_new

    def local_search(self, image, noise, loss_fct, blocks):

        # Local variables
        priority_queue = []
        num_queries = 0

        # Check if a block is in the working set or not
        A = torch.zeros((len(blocks)), dtype=torch.int32)
        for i, block in enumerate(blocks):
            upper_left, _, channel = block
            x = upper_left[0]
            y = upper_left[1]
            # If the sign of perturbation on the block is positive,
            # which means the block is in the working set, then set A to 1
            if noise[0, x, y, channel] > 0:
                A[i] = 1
        # Calculate the current loss
        image_batch = self._perturb_image(image, noise)
        correct, losses = loss_fct(image_batch, es = True)
        num_queries += 1
        curr_loss = -losses[0]
        # Early stopping
        if torch.all(correct):
            return noise, num_queries*self.EOT, curr_loss, True
        # Main loop
        for _ in range(self.max_iters):
            # Lazy greedy insert
            indices = torch.nonzero(A==0).view(-1)
            batch_size =  100
            num_batches = int(math.ceil(indices.size(0) / batch_size))
            for ibatch in range(num_batches):
                bstart = ibatch * batch_size
                bend = min(bstart + batch_size, indices.size(0))
                image_batch = torch.zeros([bend - bstart,self.img_size,self.img_size,self.in_channels]).float()
                noise_batch = torch.zeros([bend - bstart,self.img_size,self.img_size,self.in_channels]).float()
                for i, idx in enumerate(indices[bstart:bend]):
                    idx = idx.item()
                    noise_batch[i:i + 1, ...] = self._flip_noise(noise, blocks[idx])
                    image_batch[i:i + 1, ...] = self._perturb_image(image, noise_batch[i:i + 1, ...])
                correct, losses = loss_fct(image_batch, es = True)
                # Early stopping
                success_indices = torch.nonzero(correct.long()).view(-1)
                if success_indices.size(0) > 0:
                    noise[0, ...] = noise_batch[success_indices[0], ...]
                    curr_loss = -losses[success_indices[0]]
                    num_queries += success_indices[0].item() + 1
                    return noise, num_queries*self.EOT, curr_loss, True

                num_queries += bend - bstart
                # Push into the priority queue
                for i in range(bend - bstart):
                    idx = indices[bstart + i]
                    margin = -losses[i] - curr_loss
                    heapq.heappush(priority_queue, (margin.item(), idx))
            # Pick the best element and insert it into the working set
            if len(priority_queue) > 0:
                best_margin, best_idx = heapq.heappop(priority_queue)
                curr_loss += best_margin
                noise = self._flip_noise(noise, blocks[best_idx])
                A[best_idx] = 1
            # Add elements into the working set
            while len(priority_queue) > 0:
                # Pick the best element
                cand_margin, cand_idx = heapq.heappop(priority_queue)
                # Re-evalulate the element
                image_batch = self._perturb_image(
                    image, self._flip_noise(noise, blocks[cand_idx]))
                correct, losses = loss_fct(image_batch, es = True)
                num_queries += 1
                margin = -losses[0] - curr_loss
                # If the cardinality has not changed, add the element
                if len(priority_queue) == 0 or margin.item() <= priority_queue[0][0]:
                    # If there is no element that has negative margin, then break
                    if margin.item() > 0:
                        break
                    # Update the noise
                    curr_loss = -losses[0]
                    noise = self._flip_noise(noise, blocks[cand_idx])
                    A[cand_idx] = 1
                    # Early stopping
                    if torch.all(correct):
                        return noise, num_queries*self.EOT, curr_loss, True
                # If the cardinality has changed, push the element into the priority queue
                else:
                    heapq.heappush(priority_queue, (margin.item(), cand_idx))
            priority_queue = []

            # Lazy greedy delete
            indices  = torch.nonzero(A == 1).view(-1)
            batch_size = 100
            num_batches = int(math.ceil(indices.size(0) / batch_size))
            for ibatch in range(num_batches):
                bstart = ibatch * batch_size
                bend = min(bstart + batch_size, indices.size(0))

                image_batch = torch.zeros([bend - bstart, self.img_size, self.img_size,self.in_channels]).float()
                noise_batch = torch.zeros([bend - bstart, self.img_size, self.img_size,self.in_channels]).float()
                for i, idx in enumerate(indices[bstart:bend]):
                    noise_batch[i:i + 1, ...] = self._flip_noise(noise, blocks[idx])
                    image_batch[i:i + 1, ...] = self._perturb_image(image, noise_batch[i:i+1, ...])
                correct, losses = loss_fct(image_batch, es = True)
                # Early stopping
                success_indices = torch.nonzero(correct.long()).view(-1)
                if success_indices.size(0) > 0:
                    noise[0, ...] = noise_batch[success_indices[0], ...]
                    curr_loss = -losses[success_indices[0]]
                    num_queries += success_indices[0].item() + 1
                    return noise, num_queries*self.EOT, curr_loss, True
                num_queries += bend - bstart
                # Push into the priority queue
                for i in range(bend - bstart):
                    idx = indices[bstart + i].item()
                    margin = -losses[i] - curr_loss
                    heapq.heappush(priority_queue, (margin.item(), idx))

            # Pick the best element and remove it from the working set
            if len(priority_queue) > 0:
                best_margin, best_idx = heapq.heappop(priority_queue)
                curr_loss += best_margin
                noise = self._flip_noise(noise, blocks[best_idx])
                A[best_idx] = 0
            # Delete elements from the working set
            while len(priority_queue) > 0:
                # pick the best element
                cand_margin, cand_idx = heapq.heappop(priority_queue)
                # Re-evalulate the element
                image_batch = self._perturb_image(image, self._flip_noise(noise, blocks[cand_idx]))
                correct, losses = loss_fct(image_batch, es = True)
                num_queries += 1
                margin = -losses[0] - curr_loss
                # If the cardinality has not changed, remove the element
                if len(priority_queue) == 0 or margin.item() <= priority_queue[0][0]:
                    # If there is no element that has negative margin, then break
                    if margin.item() > 0:
                        break
                    # Update the noise
                    curr_loss = -losses[0]
                    noise = self._flip_noise(noise, blocks[cand_idx])
                    A[cand_idx] = 0
                    # Early stopping
                    if torch.all(correct):
                        return noise, num_queries*self.EOT, curr_loss, True
                else:
                    heapq.heappush(priority_queue, (margin.item(), cand_idx))

            priority_queue = []
        return noise, num_queries*self.EOT, curr_loss, False


    def _perturb(self, xs_t, loss_fct):

        self.img_size = xs_t.size(1)
        self.in_channels = xs_t.size(3)
        upper_left = [0, 0]
        lower_right = [self.img_size, self.img_size]
        num_queries = 0

        if self.is_new_batch:
            self.query = 0
            self.block_size = self.block_sizeo
            self.blocks = self._split_block(upper_left, lower_right, self.block_size)
            self.noise = -self.epsilon * torch.ones_like(xs_t).float()
            self.num_blocks = len(self.blocks)
            self.batch_size = self.batch_sizeo if self.batch_sizeo > 0 else self.num_blocks
            self.curr_order = torch.randperm(self.num_blocks)

        num_batches = int(math.ceil(self.num_blocks / self.batch_size))
        def loss_fct2(x, es):
            correct, loss = loss_fct(x, es = True)
            for _ in range(self.EOT - 1):
                _, new_loss = loss_fct(x, es = True)
                loss += new_loss
            return correct, loss

        for i in range(num_batches):
            # Pick a mini-batch
            bstart = i * self.batch_size
            bend = min(bstart + self.batch_size, self.num_blocks)
            blocks_batch = [self.blocks[self.curr_order[idx].item()]
                            for idx in range(bstart, bend)]
            self.noise, queries, loss, success = self.local_search(xs_t, self.noise, loss_fct2, blocks_batch)
            num_queries += queries
            self.query += queries
            print("Block size: {}, batch: {}, loss: {:.4f}, num queries: {}".format(self.block_size, i, -loss, self.query))
            # If query count exceeds the maximum queries, then return False
            if self.query > self.max_loss_queries:
                return torch.clamp(xs_t + self.noise, 0, self.ub), num_queries
            if success:
                return torch.clamp(xs_t + self.noise, 0, self.ub), num_queries

        # If block size is >= 2, then split the image into smaller blocks and reconstruct a batch
        if not self.no_hier and self.block_size >= 2:
            self.block_size //= 2
            self.blocks = self._split_block(upper_left, lower_right, self.block_size)
            self.num_blocks = len(self.blocks)
            self.batch_size = self.batch_sizeo if self.batch_sizeo > 0 else self.num_blocks
            self.curr_order = torch.randperm(self.num_blocks)
        # Otherwise, shuffle the order of the batch
        else:
            self.curr_order = torch.randperm(self.num_blocks)

        return torch.clamp(xs_t + self.noise, 0, self.ub), num_queries


    def _config(self):
        return {
            "name": self.name,
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "attack_name": self.__class__.__name__
        }


Overwriting /content/BlackboxBench/attacks/score/parsimonious_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/sign_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch

from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step, sign, norm


class SignAttack(ScoreBlackBoxAttack):
    """
    SignHunter
    """

    def __init__(self, max_loss_queries, epsilon, p, fd_eta, lb, ub, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param fd_eta: forward difference step
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size= batch_size,
                         name="Sign")


        self.fd_eta = fd_eta
        self.best_est_deriv = None
        self.xo_t = None
        self.sgn_t = None
        self.h = 0
        self.i = 0

    def _perturb(self, xs_t, loss_fct):
        _shape = list(xs_t.shape)
        dim = np.prod(_shape[1:])
        # additional queries at the start
        add_queries = 0
        if self.is_new_batch:
            self.xo_t = xs_t.clone()
            self.h = 0
            self.i = 0
        if self.i == 0 and self.h == 0:
            self.sgn_t = sign(torch.ones(_shape[0], dim))
            fxs_t = lp_step(self.xo_t, self.sgn_t.view(_shape), self.epsilon, self.p)
            bxs_t = self.xo_t
            est_deriv = (loss_fct(fxs_t) - loss_fct(bxs_t)) / self.epsilon
            self.best_est_deriv = est_deriv
            add_queries = 3  # because of bxs_t and the 2 evaluations in the i=0, h=0, case.
        chunk_len = np.ceil(dim / (2 ** self.h)).astype(int)
        istart = self.i * chunk_len
        iend = min(dim, (self.i + 1) * chunk_len)
        self.sgn_t[:, istart:iend] *= - 1.
        fxs_t = lp_step(self.xo_t, self.sgn_t.view(_shape), self.epsilon, self.p)
        bxs_t = self.xo_t
        est_deriv = (loss_fct(fxs_t) - loss_fct(bxs_t)) / self.epsilon

        ### sign here
        self.sgn_t[[i for i, val in enumerate(est_deriv < self.best_est_deriv) if val], istart: iend] *= -1.


        self.best_est_deriv = (est_deriv >= self.best_est_deriv) * est_deriv + (
                est_deriv < self.best_est_deriv) * self.best_est_deriv
        # perform the step
        new_xs = lp_step(self.xo_t, self.sgn_t.view(_shape), self.epsilon, self.p)
        # update i and h for next iteration
        self.i += 1
        if self.i == 2 ** self.h or iend == dim:
            self.h += 1
            self.i = 0
            # if h is exhausted, set xo_t to be xs_t
            if self.h == np.ceil(np.log2(dim)).astype(int) + 1:
                self.xo_t = xs_t.clone()
                self.h = 0
                print("new change")

#         if self.p == '2':
#             import pdb
#             pdb.set_trace()

        return new_xs, torch.ones(_shape[0]) + add_queries

    def get_gs(self):
        """
        return the current estimated of the gradient sign
        :return:
        """
        return self.sgn_t

    def _config(self):
        return {
            "name": self.name,
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "fd_eta": self.fd_eta,
            "attack_name": self.__class__.__name__
        }

'''

Original License

MIT License
Copyright (c) 2019 Abdullah Al-Dujaili
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/score/sign_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/simple_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch

from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step, sign


class SimpleAttack(ScoreBlackBoxAttack):
    """
    Simple Black-Box Attack
    """

    def __init__(self, max_loss_queries, epsilon, p, lb, ub, delta, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param fd_eta: forward difference step
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size = batch_size,
                         name = "SimBA")

        #self.xo_t = None
        self.delta = delta
        self.perm = None
        self.best_loss = None
        self.i = 0

    def _perturb(self, xs_t, loss_fct):
        _shape = list(xs_t.shape)
        dim = np.prod(_shape[1:])
        b_sz = _shape[0]
        add_queries = 0
        if self.is_new_batch:
            #self.xo_t = xs_t.clone()
            self.i = 0
            self.perm = torch.rand(b_sz, dim).argsort(dim=1)
        if self.i == 0:
            #self.sgn_t = sign(torch.ones(_shape[0], dim))
            #fxs_t = lp_step(self.xo_t, self.sgn_t.view(_shape), self.epsilon, self.p)
            #bxs_t = self.xo_t
            loss = loss_fct(xs_t)
            self.best_loss = loss
            add_queries = 1
        diff = torch.zeros(b_sz, dim)
        # % if iterations are greater than dim
        idx = self.perm[:, self.i % dim]
        diff = diff.scatter(1, idx.unsqueeze(1), 1)
        new_xs = xs_t.clone().contiguous().view(b_sz,-1)
        # left attempt
        left_xs = lp_step(xs_t, diff.view_as(xs_t), self.delta, self.p)
        left_loss = loss_fct(left_xs)
        replace_flag = torch.tensor((left_loss > self.best_loss).to(torch.float32)).unsqueeze(1)
        #print(replace_flag.shape)
        self.best_loss = replace_flag.squeeze(1) * left_loss + (1 - replace_flag.squeeze(1)) * self.best_loss
        new_xs = replace_flag * left_xs.contiguous().view(b_sz,-1) + (1. - replace_flag) * new_xs
        # right attempt
        right_xs = lp_step(xs_t, diff.view_as(xs_t), - self.delta, self.p)
        right_loss = loss_fct(right_xs)
        # replace only those that have greater right loss and was not replaced
        # in the left attempt
        replace_flag = torch.tensor((right_loss > self.best_loss).to(torch.float32)).unsqueeze(1) * (1 - replace_flag)
        #print(replace_flag.shape)
        self.best_loss = replace_flag.squeeze(1) * right_loss + (1 - replace_flag.squeeze(1)) * self.best_loss
        new_xs = replace_flag * right_xs.contiguous().view(b_sz,-1) + (1 - replace_flag) * new_xs
        self.i += 1
        # number of queries: add_queries (if first iteration to init best_loss) + left queries + (right queries if any)
        num_queries = add_queries + torch.ones(b_sz) + torch.ones(b_sz) * replace_flag.squeeze(1)
        return new_xs.view_as(xs_t), num_queries

    def _config(self):
        return {
            "name": self.name,
            "p": self.p,
            "epsilon": self.epsilon,
            "delta": self.delta,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "attack_name": self.__class__.__name__
        }

'''

MIT License

Copyright (c) 2019 Abdullah Al-Dujaili

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/score/simple_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/square_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from torch import Tensor as t
import torch

from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step


class SquareAttack(ScoreBlackBoxAttack):
    """
    Square Attack
    """

    def __init__(self, max_loss_queries, epsilon, p, p_init, lb, ub, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size= batch_size,
                         name = "Square")

        self.best_loss = None
        self.i = 0
        self.p_init = p_init

    def p_selection(self, p_init, it, n_iters):
        """ Piece-wise constant schedule for p (the fraction of pixels changed on every iteration). """
        it = int(it / n_iters * 10000)

        if 10 < it <= 50:
            p = p_init / 2
        elif 50 < it <= 200:
            p = p_init / 4
        elif 200 < it <= 500:
            p = p_init / 8
        elif 500 < it <= 1000:
            p = p_init / 16
        elif 1000 < it <= 2000:
            p = p_init / 32
        elif 2000 < it <= 4000:
            p = p_init / 64
        elif 4000 < it <= 6000:
            p = p_init / 128
        elif 6000 < it <= 8000:
            p = p_init / 256
        elif 8000 < it <= 10000:
            p = p_init / 512
        else:
            p = p_init

        return p

    def pseudo_gaussian_pert_rectangles(self, x, y):
        delta = torch.zeros([x, y])
        x_c, y_c = x // 2 + 1, y // 2 + 1

        counter2 = [x_c - 1, y_c - 1]
        for counter in range(0, max(x_c, y_c)):
            delta[max(counter2[0], 0):min(counter2[0] + (2 * counter + 1), x),
                max(0, counter2[1]):min(counter2[1] + (2 * counter + 1), y)] += 1.0 / (counter + 1) ** 2

            counter2[0] -= 1
            counter2[1] -= 1

        delta /= torch.sqrt(torch.sum(delta ** 2, dim=1, keepdim=True))
        return delta


    def meta_pseudo_gaussian_pert(self, s):
        delta = torch.zeros([s, s])
        n_subsquares = 2
        if n_subsquares == 2:
            delta[:s // 2] = self.pseudo_gaussian_pert_rectangles(s // 2, s)
            delta[s // 2:] = self.pseudo_gaussian_pert_rectangles(s - s // 2, s) * (-1)
            delta /= torch.sqrt(torch.sum(delta ** 2, dim=1, keepdim=True))
            if np.random.rand(1) > 0.5: delta = torch.transpose(delta, 0, 1)

        elif n_subsquares == 4:
            delta[:s // 2, :s // 2] = self.pseudo_gaussian_pert_rectangles(s // 2, s // 2) * np.random.choice([-1, 1])
            delta[s // 2:, :s // 2] = self.pseudo_gaussian_pert_rectangles(s - s // 2, s // 2) * np.random.choice([-1, 1])
            delta[:s // 2, s // 2:] = self.pseudo_gaussian_pert_rectangles(s // 2, s - s // 2) * np.random.choice([-1, 1])
            delta[s // 2:, s // 2:] = self.pseudo_gaussian_pert_rectangles(s - s // 2, s - s // 2) * np.random.choice([-1, 1])
            delta /= torch.sqrt(torch.sum(delta ** 2, dim=1, keepdim=True))

        return delta

    def _perturb(self, xs_t, loss_fct):

        xs = xs_t.permute(0,3,1,2)
        c, h, w = xs.shape[1:]
        n_features = c*h*w
        n_queries = torch.zeros(xs.shape[0])

        if self.p == 'inf':
            if self.is_new_batch:
                self.x = xs.clone()
                init_delta = t(np.random.choice([-self.epsilon, self.epsilon], size=[xs.shape[0], c, 1, w]))
                xs = torch.clamp(xs + init_delta, self.lb, self.ub)
                self.best_loss = loss_fct(xs.permute(0,2,3,1))
                n_queries += torch.ones(xs.shape[0])
                self.i = 0

            deltas = xs - self.x
            p = self.p_selection(self.p_init, self.i, 10000)
            for i_img in range(xs.shape[0]):
                s = int(round(np.sqrt(p * n_features / c)))
                s = min(max(s, 1), h-1)  # at least c x 1 x 1 window is taken and at most c x h-1 x h-1
                center_h = np.random.randint(0, h - s)
                center_w = np.random.randint(0, w - s)

                x_window = self.x[i_img, :, center_h:center_h+s, center_w:center_w+s]
                x_best_window = xs[i_img, :, center_h:center_h+s, center_w:center_w+s]
                # prevent trying out a delta if it doesn't change x_curr (e.g. an overlapping patch)
                while torch.sum(torch.abs(torch.clamp(x_window + deltas[i_img, :, center_h:center_h+s, center_w:center_w+s], self.lb, self.ub) - x_best_window) < 10**-7) == c*s*s:
                    deltas[i_img, :, center_h:center_h+s, center_w:center_w+s] = t(np.random.choice([-self.epsilon, self.epsilon], size=[c, 1, 1]))

            x_new = torch.clamp(self.x + deltas, self.lb, self.ub).permute(0,2,3,1)

        elif self.p == '2':
            if self.is_new_batch:
                self.x = xs.clone()
                delta_init = torch.zeros(xs.shape)
                s = h // 5
                sp_init = (h - s * 5) // 2
                center_h = sp_init + 0
                for _ in range(h // s):
                    center_w = sp_init + 0
                    for _ in range(w // s):
                        delta_init[:, :, center_h:center_h + s, center_w:center_w + s] += self.meta_pseudo_gaussian_pert(s).reshape(
                            [1, 1, s, s]) * t(np.random.choice([-1, 1], size=[xs.shape[0], c, 1, 1]))
                        center_w += s
                    center_h += s
                xs = torch.clamp(xs + delta_init / torch.sqrt(torch.sum(delta_init ** 2, dim=(1, 2, 3), keepdim=True)) * (self.epsilon), self.lb, self.ub)
                self.best_loss = loss_fct(xs.permute(0,2,3,1))
                n_queries += torch.ones(xs.shape[0])
                self.i = 0

            deltas = xs - self.x
            p = self.p_selection(self.p_init, self.i, 10000)
            s = max(int(round(np.sqrt(p * n_features / c))), 3)
            if s % 2 == 0:
                s += 1

            s2 = s + 0
            ### window_1
            center_h = np.random.randint(0, h - s)
            center_w = np.random.randint(0, w - s)
            new_deltas_mask = torch.zeros(xs.shape)
            new_deltas_mask[:, :, center_h:center_h + s, center_w:center_w + s] = 1.0

            ### window_2
            center_h_2 = np.random.randint(0, h - s2)
            center_w_2 = np.random.randint(0, w - s2)
            new_deltas_mask_2 = torch.zeros(xs.shape)
            new_deltas_mask_2[:, :, center_h_2:center_h_2 + s2, center_w_2:center_w_2 + s2] = 1.0

            ### compute total norm available
            curr_norms_window = torch.sqrt(
                torch.sum(((xs - self.x) * new_deltas_mask) ** 2, dim=(2, 3), keepdim=True))
            curr_norms_image = torch.sqrt(torch.sum((xs - self.x) ** 2, dim=(1, 2, 3), keepdim=True))
            mask_2 = torch.max(new_deltas_mask, new_deltas_mask_2)
            norms_windows = torch.sqrt(torch.sum((deltas * mask_2) ** 2, dim=(2, 3), keepdim=True))

            ### create the updates
            new_deltas = torch.ones([self.x.shape[0], c, s, s])
            new_deltas = new_deltas * self.meta_pseudo_gaussian_pert(s).reshape([1, 1, s, s])
            new_deltas *= t(np.random.choice([-1, 1], size=[self.x.shape[0], c, 1, 1]))
            old_deltas = deltas[:, :, center_h:center_h + s, center_w:center_w + s] / (1e-10 + curr_norms_window)
            new_deltas += old_deltas
            new_deltas = new_deltas / torch.sqrt(torch.sum(new_deltas ** 2, dim=(2, 3), keepdim=True)) * (
                torch.max((self.epsilon) ** 2 - curr_norms_image ** 2, torch.zeros_like(curr_norms_image)) / c + norms_windows ** 2) ** 0.5
            deltas[:, :, center_h_2:center_h_2 + s2, center_w_2:center_w_2 + s2] = 0.0  # set window_2 to 0
            deltas[:, :, center_h:center_h + s, center_w:center_w + s] = new_deltas + 0  # update window_1

            x_new = self.x + deltas / torch.sqrt(torch.sum(deltas ** 2, dim=(1, 2, 3), keepdim=True)) * (self.epsilon)
            x_new = torch.clamp(x_new, self.lb, self.ub).permute(0,2,3,1)


        new_loss = loss_fct(x_new)
        n_queries += torch.ones(xs.shape[0])
        idx_improved = new_loss > self.best_loss
        self.best_loss = idx_improved * new_loss + ~idx_improved * self.best_loss
        xs = xs.permute(0,2,3,1)
        idx_improved = torch.reshape(idx_improved, [-1, *[1]*len(x_new.shape[:-1])])
        x_new = idx_improved * x_new + ~idx_improved * xs
        self.i += 1

        return x_new, n_queries

    def _config(self):
        return {
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "attack_name": self.__class__.__name__
        }

'''

Copyright (c) 2019, Maksym Andriushchenko, Francesco Croce, Nicolas Flammarion, Matthias Hein
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:
    * Redistributions of source code must retain the above copyright
      notice, this list of conditions and the following disclaimer.
    * Redistributions in binary form must reproduce the above copyright
      notice, this list of conditions and the following disclaimer in the
      documentation and/or other materials provided with the distribution.
    * Neither the name of the copyright holder nor the
      names of its contributors may be used to endorse or promote products
      derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER BE LIABLE FOR ANY
DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

'''


Overwriting /content/BlackboxBench/attacks/score/square_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/score/zo_sign_sgd_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
from torch import Tensor as t
import pdb


from attacks.score.score_black_box_attack import ScoreBlackBoxAttack
from utils.compute import lp_step


class ZOSignSGDAttack(ScoreBlackBoxAttack):
    """
    ZOSignSGD Attack
    """

    def __init__(self, max_loss_queries, epsilon, p, fd_eta, lr, q, lb, ub, batch_size, name):
        """
        :param max_loss_queries: maximum number of calls allowed to loss oracle per data pt
        :param epsilon: radius of lp-ball of perturbation
        :param p: specifies lp-norm  of perturbation
        :param fd_eta: forward difference step
        :param lr: learning rate of NES step
        :param q: number of noise samples per NES step
        :param lb: data lower bound
        :param ub: data upper bound
        """
        super().__init__(max_extra_queries=np.inf,
                         max_loss_queries=max_loss_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size=batch_size,
                         name = "zosignsgd")
        self.q = q
        self.fd_eta = fd_eta
        self.lr = lr

    def _perturb(self, xs_t, loss_fct):
        #pdb.set_trace()
        _shape = list(xs_t.shape)
        dim = np.prod(_shape[1:])
        num_axes = len(_shape[1:])
        gs_t = torch.zeros_like(xs_t)
        for _ in range(self.q):
            # exp_noise = torch.randn_like(xs_t) / (dim ** 0.5)
            exp_noise = torch.randn_like(xs_t)
            fxs_t = xs_t + self.fd_eta * exp_noise
            bxs_t = xs_t
            est_deriv = (loss_fct(fxs_t) - loss_fct(bxs_t)) / self.fd_eta
            gs_t += est_deriv.reshape(-1, *[1] * num_axes) * exp_noise
        # perform the sign step regardless of the lp-ball constraint
        # this is the main difference in the method.
        new_xs = lp_step(xs_t, gs_t, self.lr, 'inf')
        # the number of queries required for forward difference is q (forward sample) + 1 at xs_t
        return new_xs, (self.q + 1) * torch.ones(_shape[0])

    def _config(self):
        return {
            'name': self.name,
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "max_extra_queries": "inf" if np.isinf(self.max_extra_queries) else self.max_extra_queries,
            "max_loss_queries": "inf" if np.isinf(self.max_loss_queries) else self.max_loss_queries,
            "lr": self.lr,
            "q": self.q,
            "fd_eta": self.fd_eta,
            "attack_name": self.__class__.__name__
        }

'''

MIT License

Copyright (c) 2019 Abdullah Al-Dujaili

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/score/zo_sign_sgd_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/decision/decision_black_box_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
from torch import Tensor as t

import sys

class DecisionBlackBoxAttack(object):
    def __init__(self, max_queries=np.inf, epsilon=0.5, p='inf', lb=0., ub=1., batch_size=1):
        """
        :param max_queries: max number of calls to model per data point
        :param epsilon: perturbation limit according to lp-ball
        :param p: norm for the lp-ball constraint
        :param lb: minimum value data point can take in any coordinate
        :param ub: maximum value data point can take in any coordinate
        """
        assert p in ['inf', '2'], "L-{} is not supported".format(p)

        self.p = p
        self.max_queries = max_queries
        self.total_queries = 0
        self.total_successes = 0
        self.total_failures = 0
        self.total_distance = 0
        self.sigma = 0
        self.EOT = 1
        self.lb = lb
        self.ub = ub
        self.epsilon = epsilon / ub
        self.batch_size = batch_size
        self.list_loss_queries = torch.zeros(1, self.batch_size)

    def result(self):
        """
        returns a summary of the attack results (to be tabulated)
        :return:
        """
        list_loss_queries = self.list_loss_queries[1:].view(-1)
        mask = list_loss_queries > 0
        list_loss_queries = list_loss_queries[mask]
        self.total_queries = int(self.total_queries)
        self.total_successes = int(self.total_successes)
        self.total_failures = int(self.total_failures)
        return {
            "total_queries": self.total_queries,
            "total_successes": self.total_successes,
            "total_failures": self.total_failures,
            "average_num_queries": "NaN" if self.total_successes == 0 else self.total_queries / self.total_successes,
            "failure_rate": "NaN" if self.total_successes + self.total_failures == 0 else self.total_failures / (self.total_successes + self.total_failures),
            "median_num_loss_queries": "NaN" if self.total_successes == 0 else torch.median(list_loss_queries).item(),
            "config": self._config()
        }

    def _config(self):
        """
        return the attack's parameter configurations as a dict
        :return:
        """
        raise NotImplementedError

    def distance(self, x_adv, x = None):
        if x is None:
            diff = x_adv.reshape(x_adv.size(0), -1)
        else:
            diff = (x_adv - x).reshape(x.size(0), -1)
        if self.p == '2':
            out = torch.sqrt(torch.sum(diff * diff)).item()
        elif self.p == 'inf':
            out = torch.sum(torch.max(torch.abs(diff), 1)[0]).item()
        return out

    def is_adversarial(self, x, y):
        '''
        check whether the adversarial constrain holds for x
        '''
        if self.targeted:
            return self.predict_label(x) == y
        else:
            return self.predict_label(x) != y

    def predict_label(self, xs):
        if type(xs) is torch.Tensor:
            x_eval = xs.permute(0,3,1,2)
        else:
            x_eval = torch.FloatTensor(xs.transpose(0,3,1,2))
        x_eval = torch.clamp(x_eval, 0, 1)
        # x_eval = x_eval + self.sigma * torch.randn_like(x_eval)
        ### Defense ###
        x_eval = self.defense_function(self.model,x_eval)
        ### Defense ###
        if self.ub == 255:
            out = self.model(x_eval)
        else:
            out = self.model(x_eval)
        l = out.argmax(dim=1)
        return l.detach()

    def _perturb(self, xs_t, ys):
        raise NotImplementedError

    def run(self, xs, ys_t, model, targeted, dset,activate_defense_function):
        self.defense_function = activate_defense_function
        self.model = model
        self.targeted = targeted
        self.train_dataset = dset

        self.logs = {
            'iteration': [0],
            'query_count': [0]
        }

        xs = xs / self.ub
        xs_t = t(xs)

        # initialize
        if self.targeted:
            check = self.is_adversarial(xs_t, ys_t)
            if torch.any(check):
                print('Some original images already belong to the target class!')
                return self.logs
        else:
            check = self.is_adversarial(xs_t, ys_t)
            if torch.any(check):
                print('Some original images do not belong to the original class!')
                return self.logs

        adv, q = self._perturb(xs_t, ys_t)
        success = self.distance(adv,xs_t) < self.epsilon
        self.total_queries += np.sum(q * success)
        self.total_successes += np.sum(success)
        self.total_failures += ys_t.shape[0] - success
        self.list_loss_queries = torch.cat([self.list_loss_queries, torch.zeros(1, self.batch_size)], dim=0)
        if type(q) is np.ndarray:
            self.list_loss_queries[-1] = t(q * success)
        else:
            self.list_loss_queries[-1] = q * success
        # self.total_distance += self.distance(adv,xs_t)

        return self.logs
'''

MIT License
Copyright (c) 2019 Abdullah Al-Dujaili
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/decision/decision_black_box_attack.py


In [ ]:
%%writefile /usr/local/lib/python3.10/dist-packages/advertorch/attacks/fast_adaptive_boundary.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
import time

try:
    from torch import flip
except ImportError:
    from advertorch.utils import torch_flip as flip

from advertorch.utils import replicate_input

from .base import Attack
from .base import LabelMixin

DEFAULT_EPS_DICT_BY_NORM = {'Linf': .3, 'L2': 1., 'L1': 5.0}


class FABAttack(Attack, LabelMixin):
    """
    Fast Adaptive Boundary Attack (Linf, L2, L1)
    https://arxiv.org/abs/1907.02044

    :param predict:       forward pass function
    :param norm:          Lp-norm to minimize ('Linf', 'L2', 'L1' supported)
    :param n_restarts:    number of random restarts
    :param n_iter:        number of iterations
    :param eps:           epsilon for the random restarts
    :param alpha_max:     alpha_max
    :param eta:           overshooting
    :param beta:          backward step
    :param device:        device to use ('cuda' or 'cpu')
    """

    def __init__(
            self,
            predict,
            norm='Linf',
            n_restarts=1,
            n_iter=100,
            eps=None,
            alpha_max=0.1,
            eta=1.05,
            beta=0.9,
            loss_fn=None,
            verbose=False,
    ):
        """ FAB-attack implementation in pytorch """

        super(FABAttack, self).__init__(
            predict, loss_fn=None, clip_min=0., clip_max=1.)

        self.norm = norm
        self.n_restarts = n_restarts
        self.n_iter = n_iter
        self.eps = eps if eps is not None else DEFAULT_EPS_DICT_BY_NORM[norm]
        self.alpha_max = alpha_max
        self.eta = eta
        self.beta = beta
        self.targeted = False
        self.verbose = verbose

    def check_shape(self, x):
        return x if len(x.shape) > 0 else x.unsqueeze(0)

    def get_diff_logits_grads_batch(self, imgs, la):
        im = imgs.clone().requires_grad_()
        with torch.enable_grad():
            y = self.predict(im)

        g2 = torch.zeros([y.shape[-1], *imgs.size()]).to(self.device)
        grad_mask = torch.zeros_like(y)
        for counter in range(y.shape[-1]):
            zero_gradients(im)
            grad_mask[:, counter] = 1.0
            y.backward(grad_mask, retain_graph=True)
            grad_mask[:, counter] = 0.0
            g2[counter] = im.grad.data

        g2 = torch.transpose(g2, 0, 1).detach()
        y2 = self.predict(imgs).detach()
        df = y2 - y2[torch.arange(imgs.shape[0]), la].unsqueeze(1)
        dg = g2 - g2[torch.arange(imgs.shape[0]), la].unsqueeze(1)
        df[torch.arange(imgs.shape[0]), la] = 1e10

        return df, dg

    def projection_linf(self, points_to_project, w_hyperplane, b_hyperplane):
        t = points_to_project.clone()
        w = w_hyperplane.clone()
        b = b_hyperplane.clone()

        ind2 = ((w * t).sum(1) - b < 0).nonzero().squeeze()
        ind2 = self.check_shape(ind2)
        w[ind2] *= -1
        b[ind2] *= -1

        c5 = (w < 0).float()
        a = torch.ones(t.shape).to(self.device)
        d = (a * c5 - t) * (w != 0).float()
        a -= a * (1 - c5)

        p = torch.ones(t.shape).to(self.device) * c5 - t * (2 * c5 - 1)
        _, indp = torch.sort(p, dim=1)

        b = b - (w * t).sum(1)
        b0 = (w * d).sum(1)
        b1 = b0.clone()

        counter = 0
        indp2 = flip(indp.unsqueeze(-1), dims=(1, 2)).squeeze()
        u = torch.arange(0, w.shape[0])
        ws = w[u.unsqueeze(1), indp2]
        bs2 = - ws * d[u.unsqueeze(1), indp2]

        s = torch.cumsum(ws.abs(), dim=1)
        sb = torch.cumsum(bs2, dim=1) + b0.unsqueeze(1)

        c = b - b1 > 0
        b2 = sb[u, -1] - s[u, -1] * p[u, indp[u, 0]]
        c_l = (b - b2 > 0).nonzero().squeeze()
        c2 = ((b - b1 > 0) * (b - b2 <= 0)).nonzero().squeeze()
        c_l = self.check_shape(c_l)
        c2 = self.check_shape(c2)

        lb = torch.zeros(c2.shape[0])
        ub = torch.ones(c2.shape[0]) * (w.shape[1] - 1)
        nitermax = torch.ceil(torch.log2(torch.tensor(w.shape[1]).float()))
        counter2 = torch.zeros(lb.shape).long()

        while counter < nitermax:
            counter4 = torch.floor((lb + ub) / 2)
            counter2 = counter4.long()
            indcurr = indp[c2, -counter2 - 1]
            b2 = sb[c2, counter2] - s[c2, counter2] * p[c2, indcurr]
            c = b[c2] - b2 > 0
            ind3 = c.nonzero().squeeze()
            ind32 = (~c).nonzero().squeeze()
            ind3 = self.check_shape(ind3)
            ind32 = self.check_shape(ind32)
            lb[ind3] = counter4[ind3]
            ub[ind32] = counter4[ind32]
            counter += 1

        lb = lb.long()
        counter2 = 0

        if c_l.nelement() != 0:
            lmbd_opt = (torch.max((b[c_l] - sb[c_l, -1]) / (-s[c_l, -1]),
                                  torch.zeros(sb[c_l, -1].shape)
                                  .to(self.device))).unsqueeze(-1)
            d[c_l] = (2 * a[c_l] - 1) * lmbd_opt

        lmbd_opt = (torch.max((b[c2] - sb[c2, lb]) / (-s[c2, lb]),
                              torch.zeros(sb[c2, lb].shape)
                              .to(self.device))).unsqueeze(-1)
        d[c2] = torch.min(lmbd_opt, d[c2]) * c5[c2]\
            + torch.max(-lmbd_opt, d[c2]) * (1 - c5[c2])

        return d * (w != 0).float()

    def projection_l2(self, points_to_project, w_hyperplane, b_hyperplane):
        t = points_to_project.clone()
        w = w_hyperplane.clone()
        b = b_hyperplane.clone()

        c = (w * t).sum(1) - b
        ind2 = (c < 0).nonzero().squeeze()
        ind2 = self.check_shape(ind2)
        w[ind2] *= -1
        c[ind2] *= -1

        u = torch.arange(0, w.shape[0]).unsqueeze(1)

        r = torch.max(t / w, (t - 1) / w)
        u2 = torch.ones(r.shape).to(self.device)
        r = torch.min(r, 1e12 * u2)
        r = torch.max(r, -1e12 * u2)
        r[w.abs() < 1e-8] = 1e12
        r[r == -1e12] = -r[r == -1e12]
        rs, indr = torch.sort(r, dim=1)
        rs2 = torch.cat((rs[:, 1:],
                         torch.zeros(rs.shape[0], 1).to(self.device)), 1)
        rs[rs == 1e12] = 0
        rs2[rs2 == 1e12] = 0

        w3 = w ** 2
        w3s = w3[u, indr]
        w5 = w3s.sum(dim=1, keepdim=True)
        ws = w5 - torch.cumsum(w3s, dim=1)
        d = -(r * w).clone()
        d = d * (w.abs() > 1e-8).float()
        s = torch.cat(((-w5.squeeze() * rs[:, 0]).unsqueeze(1),
                       torch.cumsum((-rs2 + rs) * ws, dim=1) -
                       w5 * rs[:, 0].unsqueeze(-1)), 1)

        c4 = (s[:, 0] + c < 0)
        c3 = ((d * w).sum(dim=1) + c > 0)
        c6 = c4.nonzero().squeeze()
        c2 = ((1 - c4.float()) * (1 - c3.float())).nonzero().squeeze()
        c6 = self.check_shape(c6)
        c2 = self.check_shape(c2)

        counter = 0
        lb = torch.zeros(c2.shape[0])
        ub = torch.ones(c2.shape[0]) * (w.shape[1] - 1)
        nitermax = torch.ceil(torch.log2(torch.tensor(w.shape[1]).float()))
        counter2 = torch.zeros(lb.shape).long()

        while counter < nitermax:
            counter4 = torch.floor((lb + ub) / 2)
            counter2 = counter4.long()
            c3 = s[c2, counter2] + c[c2] > 0
            ind3 = c3.nonzero().squeeze()
            ind32 = (~c3).nonzero().squeeze()
            ind3 = self.check_shape(ind3)
            ind32 = self.check_shape(ind32)
            lb[ind3] = counter4[ind3]
            ub[ind32] = counter4[ind32]
            counter += 1

        lb = lb.long()
        alpha = torch.zeros([1])

        if c6.nelement() != 0:
            alpha = c[c6] / w5[c6].squeeze(-1)
            d[c6] = -alpha.unsqueeze(-1) * w[c6]

        if c2.nelement() != 0:
            alpha = (s[c2, lb] + c[c2]) / ws[c2, lb] + rs[c2, lb]
            if torch.sum(ws[c2, lb] == 0) > 0:
                ind = (ws[c2, lb] == 0).nonzero().squeeze().long()
                ind = self.check_shape(ind)
                alpha[ind] = 0
            c5 = (alpha.unsqueeze(-1) > r[c2]).float()
            d[c2] = d[c2] * c5 - alpha.unsqueeze(-1) * w[c2] * (1 - c5)

        return d * (w.abs() > 1e-8).float()

    def projection_l1(self, points_to_project, w_hyperplane, b_hyperplane):
        t = points_to_project.clone()
        w = w_hyperplane.clone()
        b = b_hyperplane.clone()

        c = (w * t).sum(1) - b
        ind2 = (c < 0).nonzero().squeeze()
        ind2 = self.check_shape(ind2)
        w[ind2] *= -1
        c[ind2] *= -1

        r = torch.max(1 / w, -1 / w)
        r = torch.min(r, 1e12 * torch.ones(r.shape).to(self.device))
        rs, indr = torch.sort(r, dim=1)
        _, indr_rev = torch.sort(indr)

        u = torch.arange(0, w.shape[0]).unsqueeze(1)
        u2 = torch.arange(0, w.shape[1]).repeat(w.shape[0], 1)
        c6 = (w < 0).float()
        d = (-t + c6) * (w != 0).float()
        d2 = torch.min(-w * t, w * (1 - t))
        ds = d2[u, indr]
        ds2 = torch.cat((c.unsqueeze(-1), ds), 1)
        s = torch.cumsum(ds2, dim=1)

        c4 = s[:, -1] < 0
        c2 = c4.nonzero().squeeze(-1)
        c2 = self.check_shape(c2)

        counter = 0
        lb = torch.zeros(c2.shape[0])
        ub = torch.ones(c2.shape[0]) * (s.shape[1])
        nitermax = torch.ceil(torch.log2(torch.tensor(s.shape[1]).float()))
        counter2 = torch.zeros(lb.shape).long()

        while counter < nitermax:
            counter4 = torch.floor((lb + ub) / 2)
            counter2 = counter4.long()
            c3 = s[c2, counter2] > 0
            ind3 = c3.nonzero().squeeze()
            ind32 = (~c3).nonzero().squeeze()
            ind3 = self.check_shape(ind3)
            ind32 = self.check_shape(ind32)
            lb[ind3] = counter4[ind3]
            ub[ind32] = counter4[ind32]
            counter += 1

        lb2 = lb.long()

        if c2.nelement() != 0:
            alpha = -s[c2, lb2] / w[c2, indr[c2, lb2]]
            c5 = u2[c2].float() < lb.unsqueeze(-1).float()
            u3 = c5[u[:c5.shape[0]], indr_rev[c2]]
            d[c2] = d[c2] * u3.float().to(self.device)
            d[c2, indr[c2, lb2]] = alpha

        return d * (w.abs() > 1e-8).float()

    def perturb(self, x, y=None):
        """
        :param x:    clean images
        :param y:    clean labels, if None we use the predicted labels
        """

        self.device = x.device
        self.orig_dim = list(x.shape[1:])
        self.ndims = len(self.orig_dim)

        x = x.detach().clone().float().to(self.device)
        # assert next(self.predict.parameters()).device == x.device

        y_pred = self._get_predicted_label(x)
        if y is None:
            y = y_pred.detach().clone().long().to(self.device)
        else:
            y = y.detach().clone().long().to(self.device)
        pred = y_pred == y
        corr_classified = pred.float().sum()
        if self.verbose:
            print('Clean accuracy: {:.2%}'.format(pred.float().mean()))
        if pred.sum() == 0:
            return x
        pred = self.check_shape(pred.nonzero().squeeze())

        startt = time.time()
        # runs the attack only on correctly classified points
        im2 = replicate_input(x[pred])
        la2 = replicate_input(y[pred])
        if len(im2.shape) == self.ndims:
            im2 = im2.unsqueeze(0)
        bs = im2.shape[0]
        u1 = torch.arange(bs)
        adv = im2.clone()
        adv_c = x.clone()
        res2 = 1e10 * torch.ones([bs]).to(self.device)
        res_c = torch.zeros([x.shape[0]]).to(self.device)
        x1 = im2.clone()
        x0 = im2.clone().reshape([bs, -1])
        counter_restarts = 0

        while counter_restarts < self.n_restarts:
            if counter_restarts > 0:
                if self.norm == 'Linf':
                    t = 2 * torch.rand(x1.shape).to(self.device) - 1
                    x1 = im2 + (
                        torch.min(
                            res2,
                            self.eps * torch.ones(res2.shape).to(self.device)
                        ).reshape([-1, *([1] * self.ndims)])
                    ) * t / (t.reshape([t.shape[0], -1]).abs()
                             .max(dim=1, keepdim=True)[0]
                             .reshape([-1, *([1] * self.ndims)])) * .5
                elif self.norm == 'L2':
                    t = torch.randn(x1.shape).to(self.device)
                    x1 = im2 + (
                        torch.min(
                            res2,
                            self.eps * torch.ones(res2.shape).to(self.device)
                        ).reshape([-1, *([1] * self.ndims)])
                    ) * t / ((t ** 2)
                             .view(t.shape[0], -1)
                             .sum(dim=-1)
                             .sqrt()
                             .view(t.shape[0], *([1] * self.ndims))) * .5
                elif self.norm == 'L1':
                    t = torch.randn(x1.shape).to(self.device)
                    x1 = im2 + (torch.min(
                        res2,
                        self.eps * torch.ones(res2.shape).to(self.device)
                    ).reshape([-1, *([1] * self.ndims)])
                    ) * t / (t.abs().view(t.shape[0], -1)
                             .sum(dim=-1)
                             .view(t.shape[0], *([1] * self.ndims))) / 2

                x1 = x1.clamp(0.0, 1.0)

            counter_iter = 0
            while counter_iter < self.n_iter:
                with torch.no_grad():
                    df, dg = self.get_diff_logits_grads_batch(x1, la2)
                    if self.norm == 'Linf':
                        dist1 = df.abs() / (1e-12 +
                                            dg.abs()
                                            .view(dg.shape[0], dg.shape[1], -1)
                                            .sum(dim=-1))
                    elif self.norm == 'L2':
                        dist1 = df.abs() / (1e-12 + (dg ** 2)
                                            .view(dg.shape[0], dg.shape[1], -1)
                                            .sum(dim=-1).sqrt())
                    elif self.norm == 'L1':
                        dist1 = df.abs() / (1e-12 + dg.abs().reshape(
                            [df.shape[0], df.shape[1], -1]).max(dim=2)[0])
                    else:
                        raise ValueError('norm not supported')
                    ind = dist1.min(dim=1)[1]
                    dg2 = dg[u1, ind]
                    b = (- df[u1, ind] +
                         (dg2 * x1).view(x1.shape[0], -1).sum(dim=-1))
                    w = dg2.reshape([bs, -1])

                    if self.norm == 'Linf':
                        d3 = self.projection_linf(
                            torch.cat((x1.reshape([bs, -1]), x0), 0),
                            torch.cat((w, w), 0),
                            torch.cat((b, b), 0))
                    elif self.norm == 'L2':
                        d3 = self.projection_l2(
                            torch.cat((x1.reshape([bs, -1]), x0), 0),
                            torch.cat((w, w), 0),
                            torch.cat((b, b), 0))
                    elif self.norm == 'L1':
                        d3 = self.projection_l1(
                            torch.cat((x1.reshape([bs, -1]), x0), 0),
                            torch.cat((w, w), 0),
                            torch.cat((b, b), 0))
                    d1 = torch.reshape(d3[:bs], x1.shape)
                    d2 = torch.reshape(d3[-bs:], x1.shape)
                    if self.norm == 'Linf':
                        a0 = d3.abs().max(dim=1, keepdim=True)[0]\
                            .view(-1, *([1] * self.ndims))
                    elif self.norm == 'L2':
                        a0 = (d3 ** 2).sum(dim=1, keepdim=True).sqrt()\
                            .view(-1, *([1] * self.ndims))
                    elif self.norm == 'L1':
                        a0 = d3.abs().sum(dim=1, keepdim=True)\
                            .view(-1, *([1] * self.ndims))
                    a0 = torch.max(a0, 1e-8 * torch.ones(
                        a0.shape).to(self.device))
                    a1 = a0[:bs]
                    a2 = a0[-bs:]
                    alpha = torch.min(torch.max(a1 / (a1 + a2),
                                                torch.zeros(a1.shape)
                                                .to(self.device))[0],
                                      self.alpha_max * torch.ones(a1.shape)
                                      .to(self.device))
                    x1 = ((x1 + self.eta * d1) * (1 - alpha) +
                          (im2 + d2 * self.eta) * alpha).clamp(0.0, 1.0)

                    is_adv = self._get_predicted_label(x1) != la2

                    if is_adv.sum() > 0:
                        ind_adv = is_adv.nonzero().squeeze()
                        ind_adv = self.check_shape(ind_adv)
                        if self.norm == 'Linf':
                            t = (x1[ind_adv] - im2[ind_adv]).reshape(
                                [ind_adv.shape[0], -1]).abs().max(dim=1)[0]
                        elif self.norm == 'L2':
                            t = ((x1[ind_adv] - im2[ind_adv]) ** 2)\
                                .view(ind_adv.shape[0], -1).sum(dim=-1).sqrt()
                        elif self.norm == 'L1':
                            t = (x1[ind_adv] - im2[ind_adv])\
                                .abs().view(ind_adv.shape[0], -1).sum(dim=-1)
                        adv[ind_adv] = x1[ind_adv] * (t < res2[ind_adv]).\
                            float().reshape([-1, *([1] * self.ndims)]) \
                            + adv[ind_adv]\
                            * (t >= res2[ind_adv]).float().reshape(
                            [-1, *([1] * self.ndims)])
                        res2[ind_adv] = t * (t < res2[ind_adv]).float()\
                            + res2[ind_adv] * (t >= res2[ind_adv]).float()
                        x1[ind_adv] = im2[ind_adv] + (
                            x1[ind_adv] - im2[ind_adv]) * self.beta

                    counter_iter += 1

            counter_restarts += 1

        ind_succ = res2 < 1e10
        if self.verbose:
            print('success rate: {:.0f}/{:.0f}'
                  .format(ind_succ.float().sum(), corr_classified) +
                  ' (on correctly classified points) in {:.1f} s'
                  .format(time.time() - startt))

        res_c[pred] = res2 * ind_succ.float() + 1e10 * (1 - ind_succ.float())
        ind_succ = self.check_shape(ind_succ.nonzero().squeeze())
        adv_c[pred[ind_succ]] = adv[ind_succ].clone()

        return adv_c


class LinfFABAttack(FABAttack):
    """
    Linf - Fast Adaptive Boundary Attack
    https://arxiv.org/abs/1907.02044

    :param predict:       forward pass function
    :param n_restarts:    number of random restarts
    :param n_iter:        number of iterations
    :param eps:           epsilon for the random restarts
    :param alpha_max:     alpha_max
    :param eta:           overshooting
    :param beta:          backward step
    :param device:        device to use ('cuda' or 'cpu')
    """

    def __init__(
            self,
            predict,
            n_restarts=1,
            n_iter=100,
            eps=None,
            alpha_max=0.1,
            eta=1.05,
            beta=0.9,
            loss_fn=None,
            verbose=False,
    ):
        norm = 'Linf'
        super(LinfFABAttack, self).__init__(
            predict=predict, norm=norm, n_restarts=n_restarts,
            n_iter=n_iter, eps=eps, alpha_max=alpha_max, eta=eta, beta=beta,
            loss_fn=loss_fn, verbose=verbose)


class L2FABAttack(FABAttack):
    """
    L2 - Fast Adaptive Boundary Attack
    https://arxiv.org/abs/1907.02044

    :param predict:       forward pass function
    :param n_restarts:    number of random restarts
    :param n_iter:        number of iterations
    :param eps:           epsilon for the random restarts
    :param alpha_max:     alpha_max
    :param eta:           overshooting
    :param beta:          backward step
    :param device:        device to use ('cuda' or 'cpu')
    """

    def __init__(
            self,
            predict,
            n_restarts=1,
            n_iter=100,
            eps=None,
            alpha_max=0.1,
            eta=1.05,
            beta=0.9,
            loss_fn=None,
            verbose=False,
    ):
        norm = 'L2'
        super(L2FABAttack, self).__init__(
            predict=predict, norm=norm, n_restarts=n_restarts,
            n_iter=n_iter, eps=eps, alpha_max=alpha_max, eta=eta, beta=beta,
            loss_fn=loss_fn, verbose=verbose)


class L1FABAttack(FABAttack):
    """
    L1 - Fast Adaptive Boundary Attack
    https://arxiv.org/abs/1907.02044

    :param predict:       forward pass function
    :param n_restarts:    number of random restarts
    :param n_iter:        number of iterations
    :param eps:           epsilon for the random restarts
    :param alpha_max:     alpha_max
    :param eta:           overshooting
    :param beta:          backward step
    :param device:        device to use ('cuda' or 'cpu')
    """

    def __init__(
            self,
            predict,
            n_restarts=1,
            n_iter=100,
            eps=None,
            alpha_max=0.1,
            eta=1.05,
            beta=0.9,
            loss_fn=None,
            verbose=False,
    ):
        norm = 'L1'
        super(L1FABAttack, self).__init__(
            predict=predict, norm=norm, n_restarts=n_restarts,
            n_iter=n_iter, eps=eps, alpha_max=alpha_max, eta=eta, beta=beta,
            loss_fn=loss_fn, verbose=verbose)


Overwriting /usr/local/lib/python3.10/dist-packages/advertorch/attacks/fast_adaptive_boundary.py


In [ ]:
%%writefile /content/BlackboxBench/datasets/dataset.py

"""
A wrapper for datasets
    mnist, cifar10, imagenet
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# from tensorflow.examples.tutorials.mnist import input_data

import datasets.cifar10 as cifar10_input
from datasets.imagenet import ImagenetValidData
from utils.misc import data_path_join
import torch
import numpy as np
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

def load_imagenet(n_ex = 1000, size=224):
    IMAGENET_SL = size
    IMAGENET_PATH = data_path_join('imagenet/Sample_1000')
    imagenet = ImageFolder(IMAGENET_PATH,
                           transforms.Compose([
                               transforms.Resize(IMAGENET_SL),
                               transforms.CenterCrop(IMAGENET_SL),
                               transforms.ToTensor()
                           ]))
    torch.manual_seed(0)

    imagenet_loader = DataLoader(imagenet, batch_size=n_ex, shuffle=True, num_workers=1)
    x_test, y_test = next(iter(imagenet_loader))
    return np.array(x_test, dtype=np.float32), np.array(y_test)

class Dataset(object):
    def __init__(self, name, config):
        """
        :param name: dataset name
        :param config: dictionary whose keys are dependent on the dataset created
         (see source code below)
        """
        assert name in ['mnist', 'cifar10', 'cifar10aug', 'imagenet', 'imagenet_sub'], "Invalid dataset"
        self.name = name
        model_name = config['modeln']
        # config = config['dset_config']

        if self.name == 'cifar10':
            data_path = data_path_join('/content/BlackboxBench/data')
            self.data = cifar10_input.CIFAR10Data(data_path)
        elif self.name == 'cifar10aug':
            data_path = data_path_join('cifar10_data')
            # raw_cifar = cifar10_input.CIFAR10Data(data_path)
            # sess = config['sess']
            # model = config['model']
            # self.data = cifar10_input.AugmentedCIFAR10Data(raw_cifar, sess, model)
        elif self.name == 'mnist':
            self.data = input_data.read_data_sets(data_path_join('mnist_data'), one_hot=False)
        elif self.name == 'imagenet':
            data_path = data_path_join('imagenet_data')
            self.data = ImagenetValidData(data_dir=data_path)
        elif self.name == 'imagenet_sub':
            if model_name == 'Inception':
                self.x_test, self.y_test = load_imagenet(n_ex=1000, size=299)
            else:
                self.x_test, self.y_test = load_imagenet()
            self.x_test = self.x_test.transpose(0, 2, 3, 1)

    def get_next_train_batch(self, batch_size):
        """
        Returns a tuple of (x_batch, y_batch)
        """
        if self.name in ['cifar10', 'cifar10aug']:
            return self.data.train_data.get_next_batch(batch_size, multiple_passes=True)
        elif self.name == 'mnist':
            return self.data.train.next_batch(batch_size)
        elif self.name in ['imagenet', 'imagenet_sub']:
            raise Exception(
                'No training data for imagenet is needed (provided), the models are assumed to be pretrained!')

    def get_eval_data(self, bstart, bend):
        """
        :param bstart: batch start index
        :param bend: batch end index
        """
        if self.name in ['cifar10', 'cifar10aug']:
            return self.data.eval_data.xs[bstart:bend, :], \
                   self.data.eval_data.ys[bstart:bend]
        elif self.name == 'mnist':
            return self.data.test.images[bstart:bend, :], \
                   self.data.test.labels[bstart:bend]
        elif self.name == 'imagenet':
            return self.data.get_eval_data(bstart, bend)
        elif self.name == 'imagenet_sub':
            return self.x_test[bstart:bend, :], \
                    self.y_test[bstart:bend]


    @property
    def min_value(self):
        if self.name in ['cifar10', 'cifar10aug', 'mnist', 'imagenet', 'imagenet_sub']:
            return 0.

    @property
    def max_value(self):
        if self.name in ['cifar10', 'cifar10aug']:
            return 255.
        if self.name in ['mnist', 'imagenet', 'imagenet_sub']:
            return 1.


Overwriting /content/BlackboxBench/datasets/dataset.py


In [ ]:
%%writefile /content/BlackboxBench/utils/model_loader.py

from __future__ import division
from __future__ import absolute_import

import torch
import os
import json
import torch.nn as nn
from utils.misc import data_path_join
# from utils_.utils import RecorderMeter
# from utils_ import utils
from models import resnet, wrn, vgg
# , wrn_sap, wrn_adv_sap, model_zoo, vgg_rse, pni_model
# from models.Resnet import resnet152_denoise, resnet101_denoise
import numpy as np
from torchvision import models, datasets

def load_torch_models(model_name):

    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    if model_name == "resnet20":
        # TRAINED_MODEL_PATH = data_path_join('pretrained_models/pyramidnet_basic_110_84/00/')
        # filename = 'model_best_state.pth'
        # with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
        pretrained_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
        # pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename))['state_dict'])
    elif model_name == 'resnet50-AT':

        from robustness import cifar_models
        checkpoint = torch.load('cifar_linf_8.pt')
        # Extract the model state dictionary from the OrderedDict (replace 'model' with the correct key)
        model_state_dict = checkpoint['state_dict']
        # Create an instance of the ResNet50 model
        pretrained_model = cifar_models.resnet50()
        mapped_state_dict = {k.replace('module.attacker.model.', ''): v for k, v in model_state_dict.items()}
        # # Filter out keys that are not relevant to ResNet50
        filtered_state_dict = {k: v for k, v in mapped_state_dict.items() if k in pretrained_model.state_dict()}
        # Load the filtered state dictionary
        pretrained_model.load_state_dict(filtered_state_dict)
    elif model_name == 'vgg16':
        # TRAINED_MODEL_PATH = data_path_join('pretrained_models/resnet_adv_4/cifar-10_linf/')
        # filename = 'model_best_state.pth'
        # with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
        pretrained_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=True)
        # pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename)))
    elif model_name == 'resnet':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/resnet_basic_110/00/')
        filename = 'model_best_state.pth'
        with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
            pretrained_model = resnet.Network(json.load(fr)['model_config'])
            pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename))['state_dict'])
    elif model_name == 'wrn':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_28_10/00/')
        filename = 'model_best_state.pth'
        with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
            pretrained_model = wrn.Network(json.load(fr)['model_config'])
            pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename))['state_dict'])

    elif model_name == 'dense':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/densenet_BC_100_12/00/')
        filename = 'model_best_state.pth'
        with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
            pretrained_model = densenet.Network(json.load(fr)['model_config'])
            pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename))['state_dict'])


    elif model_name == 'vgg_rse':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/rse_model/')
        filename = 'cifar10_vgg_rse_005.pth'
        # with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
            # pretrained_model = vgg.Network(json.load(fr)['model_config'])
        noise_init = 0
        noise_inner = 0
        pretrained_model = nn.DataParallel(vgg_rse.VGG_RSE('VGG16', 10, noise_init, noise_inner, img_width=32), device_ids=range(1))
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename)))

    elif model_name == 'rse':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/rse_model/')
        filename = 'cifar10_vgg_rse.pth'
        # with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
            # pretrained_model = vgg.Network(json.load(fr)['model_config'])
        noise_init = 0.2
        noise_inner = 0.1
        pretrained_model = nn.DataParallel(vgg_rse.VGG_RSE('VGG16', 10, noise_init, noise_inner, img_width=32), device_ids=range(1))
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename)))



    elif model_name == 'vgg':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/vgg_15_BN_64/00/')
        filename = 'model_best_state.pth'
        with open(os.path.join(TRAINED_MODEL_PATH, 'config.json')) as fr:
            pretrained_model = vgg.Network(json.load(fr)['model_config'])
            pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename),map_location=device)['state_dict'])


    elif model_name == 'wrn_adv':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'adv_wrn16_linf.pth'
        pretrained_model = wrn_adv.create_model(name='wide', num_classes = 10)
        if hasattr(pretrained_model, 'module'):
            pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename),map_location=device)['state_dict'])

    elif model_name == 'wrn28':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'cifar_wrn_28.pth'
        pretrained_model = wrn.WideNet()
        pretrained_model = torch.nn.DataParallel(pretrained_model)
        checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(checkpoint['net'])

    # elif model_name == 'wrn28_sap':
    #     TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
    #     filename = 'cifar_wrn_28.pth'
    #     pretrained_model = wrn_sap.WideNet()
    #     pretrained_model = torch.nn.DataParallel(pretrained_model)
    #     checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
    #     # if hasattr(pretrained_model, 'module'):
    #     #     pretrained_model = pretrained_model.module
    #     pretrained_model.load_state_dict(checkpoint['net'])

    elif model_name == 'wrn16_clean':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'wrn_clean.pth'
        pretrained_model = wrn_adv.create_model(name='wide', num_classes = 10)
        if hasattr(pretrained_model, 'module'):
            pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename),map_location=device)['state_dict'])


    elif model_name == 'wrn16_fine':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'wrn16_fine.pth'
        pretrained_model = wrn_adv.create_model(name='wide16', num_classes = 10)
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename),map_location=device))


    elif model_name == 'wrn28_fine':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'wrn28_fine.pth'
        pretrained_model = wrn.WideNet()
        pretrained_model = torch.nn.DataParallel(pretrained_model)
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename),map_location=device))



    elif model_name == 'wrn16_clean_sap':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'wrn_clean.pth'
        pretrained_model = wrn_adv_sap.create_model(name='wide', num_classes = 10)
        if hasattr(pretrained_model, 'module'):
            pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename),map_location=device)['state_dict'])


    # elif model_name == 'wrn_gau':
    #     TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
    #     filename = 'wrn_005.pth'
    #     # pretrained_model = nn.DataParallel(adv_model.create_model(name='wide_gau', num_classes = 10), device_ids=range(1))
    #     # pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename)))
    #     pretrained_model = wrn_adv.create_model(name='wide', num_classes = 10)
    #     if hasattr(pretrained_model, 'module'):
    #         pretrained_model = pretrained_model.module
    #     pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename))['state_dict'])

    elif model_name == 'wrn_01':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = '01checkpoint_200.pth'
        # pretrained_model = nn.DataParallel(adv_model.create_model(name='wide_gau', num_classes = 10), device_ids=range(1))
        # pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename)))
        pretrained_model = wrn_adv.create_model(name='wide', num_classes = 10)
        if hasattr(pretrained_model, 'module'):
            pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(torch.load(os.path.join(TRAINED_MODEL_PATH, filename))['state_dict'])

    elif model_name == 'res18':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/resnet/')
        filename = 'cifar_res18.pth'
        pretrained_model = resnet.ResNet18()
        pretrained_model = torch.nn.DataParallel(pretrained_model)
        checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(checkpoint['net'])


    elif model_name == 'res50':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/resnet/')
        filename = 'cifar_res50.pth'
        pretrained_model = resnet.ResNet50()
        pretrained_model = torch.nn.DataParallel(pretrained_model)
        checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(checkpoint['net'])


    elif model_name == 'res101':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/resnet/')
        filename = 'cifar_res101.pth'
        pretrained_model = resnet.ResNet101()
        pretrained_model = torch.nn.DataParallel(pretrained_model)
        checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(checkpoint['net'])

    elif model_name == 'dense121':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/densenet/')
        filename = 'cifar_dense121.pth'
        pretrained_model = densenet.DenseNet121()
        pretrained_model = torch.nn.DataParallel(pretrained_model)
        checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        # if hasattr(pretrained_model, 'module'):
        #     pretrained_model = pretrained_model.module
        pretrained_model.load_state_dict(checkpoint['net'])


    elif model_name == 'best_adv_wrn34':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'cifar10_linf_wrn34-20.pt'
        pretrained_model = model_zoo.WideResNet(
            num_classes=10, depth=34, width=20,
            activation_fn=model_zoo.Swish, mean=model_zoo.CIFAR10_MEAN,
            std=model_zoo.CIFAR10_STD)
        # dataset_fn = datasets.CIFAR10
        params = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        pretrained_model.load_state_dict(params)

    elif model_name == 'best_adv_wrn28':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/wrn_adv/')
        filename = 'cifar10_linf_wrn28-10_with.pt'
        pretrained_model = model_zoo.WideResNet(
            num_classes=10, depth=28, width=10,
            activation_fn=model_zoo.Swish, mean=model_zoo.CIFAR10_MEAN,
            std=model_zoo.CIFAR10_STD)
        # dataset_fn = datasets.CIFAR10
        params = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        pretrained_model.load_state_dict(params)


    elif model_name == 'pni':
        TRAINED_MODEL_PATH = data_path_join('pretrained_models/pni_model/')
        filename = 'checkpoint.pth.tar'
        recorder = RecorderMeter(200)  # count number of epoches
        pretrained_model = pni_model.noise_resnet20()
        checkpoint = torch.load(os.path.join(TRAINED_MODEL_PATH, filename))
        state_tmp = pretrained_model.state_dict()
        if 'state_dict' in checkpoint.keys():
            state_tmp.update(checkpoint['state_dict'])
        else:
            state_tmp.update(checkpoint)

        pretrained_model.load_state_dict(state_tmp)

    def normalize_fn(tensor, mean, std):
        """Differentiable version of torchvision.functional.normalize"""
        # here we assume the color channel is in at dim=1
        mean = mean[None, :, None, None]
        std = std[None, :, None, None]
        return tensor.sub(mean).div(std)

    class NormalizeByChannelMeanStd(nn.Module):
        def __init__(self, mean, std):
            super(NormalizeByChannelMeanStd, self).__init__()
            if not isinstance(mean, torch.Tensor):
                mean = torch.tensor(mean)
            if not isinstance(std, torch.Tensor):
                std = torch.tensor(std)
            self.register_buffer("mean", mean)
            self.register_buffer("std", std)

        def forward(self, tensor):
            return normalize_fn(tensor, self.mean, self.std)

        def extra_repr(self):
            return 'mean={}, std={}'.format(self.mean, self.std)

    if  (model_name == 'vgg_plain') or (model_name == 'vgg_rse') or (model_name == 'rse') or (model_name == 'pni') or (model_name == 'wrn_adv') or (model_name == 'wrn16_clean') or (model_name == 'wrn16_fine') or (model_name == 'wrn16_clean_sap') or (model_name == 'wrn_01') or (model_name == 'best_adv_wrn28') or (model_name == 'best_adv_wrn34') or (model_name == 'wrn_stop') or (model_name == 'sat_wrn16_l2') or (model_name == 'sat_wrn16_linf'):
        net = pretrained_model

    elif 'wrn28' in model_name:
        mean = np.array([0.4914, 0.4822, 0.4465])
        std = np.array([0.2023, 0.1994, 0.2010])

        # std = np.array([0.2470, 0.2435, 0.2616])
        normalize = NormalizeByChannelMeanStd(
                mean=mean.tolist(), std=std.tolist())

        net = nn.Sequential(
            normalize,
            pretrained_model
        )

    else:
        mean = np.array([0.4914, 0.4822, 0.4465])
        std = np.array([0.2470, 0.2435, 0.2616])

        normalize = NormalizeByChannelMeanStd(
                mean=mean.tolist(), std=std.tolist())

        net = nn.Sequential(
            normalize,
            pretrained_model
        )

    if torch.cuda.is_available():
        net = net.cuda()
    net.eval()
    return net


class Permute(nn.Module):

    def __init__(self, permutation = [2,1,0]):
        super().__init__()
        self.permutation = permutation

    def forward(self, input):

        return input[:, self.permutation]


def load_torch_models_imagesub(model_name):
    if model_name == "VGG16":
        pretrained_model = models.vgg16_bn(pretrained=True)
    elif model_name == 'Resnet18':
        pretrained_model = models.resnet18(pretrained=True)
    elif model_name == 'Resnet34':
        pretrained_model = models.resnet34(pretrained=True)
    elif model_name == 'Resnet50':
        pretrained_model = models.resnet50(pretrained=True)
    elif model_name == 'resnet50-AT':
        checkpoint = torch.load('imagenet_linf_8.pt')
        # Extract the model state dictionary from the OrderedDict (replace 'model' with the correct key)
        model_state_dict = checkpoint['model']
        # Create an instance of the ResNet50 model
        pretrained_model = models.resnet50(pretrained=True)
        mapped_state_dict = {k.replace('module.attacker.model.', ''): v for k, v in model_state_dict.items()}
        # Filter out keys that are not relevant to ResNet50
        filtered_state_dict = {k: v for k, v in mapped_state_dict.items() if k in pretrained_model.state_dict()}
        # Load the filtered state dictionary
        pretrained_model.load_state_dict(filtered_state_dict, strict=False)
    elif model_name == 'Resnet101':
        pretrained_model = models.resnet101(pretrained=True)
    # elif model_name == 'Squeezenet':
    #     pretrained_model = models.squeezenet1_1(pretrained=True)
    elif model_name == 'Googlenet':
        pretrained_model = models.googlenet(pretrained=True)
    elif model_name == 'Inception':
        pretrained_model = models.inception_v3(pretrained=True)
    elif model_name == 'Widenet':
        pretrained_model = models.wide_resnet50_2(pretrained=True)
    # elif model_name == 'Adv_Denoise_Resnext101':
    #     pretrained_model = resnet101_denoise()
    #     loaded_state_dict = torch.load(os.path.join('./results/denoise/', model_name+".pytorch"))
    #     pretrained_model.load_state_dict(loaded_state_dict)

    # if 'defense' in state and state['defense']:
    #     net = nn.Sequential(
    #         Normalize(mean, std),
    #         Permute([2,1,0]),
    #         pretrained_model
    #     )
    # else:
    # net = nn.Sequential(
    #     Normalize(mean, std),
    #     pretrained_model
    # )

    # from advertorch.utils import NormalizeByChannelMeanStd
    def normalize_fn(tensor, mean, std):
        """Differentiable version of torchvision.functional.normalize"""
        # here we assume the color channel is in at dim=1
        mean = mean[None, :, None, None]
        std = std[None, :, None, None]
        return tensor.sub(mean).div(std)
    class NormalizeByChannelMeanStd(nn.Module):
        def __init__(self, mean, std):
            super(NormalizeByChannelMeanStd, self).__init__()
            if not isinstance(mean, torch.Tensor):
                mean = torch.tensor(mean)
            if not isinstance(std, torch.Tensor):
                std = torch.tensor(std)
            self.register_buffer("mean", mean)
            self.register_buffer("std", std)

        def forward(self, tensor):
            return normalize_fn(tensor, self.mean, self.std)

        def extra_repr(self):
            return 'mean={}, std={}'.format(self.mean, self.std)

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    normalize = NormalizeByChannelMeanStd(
            mean=mean.tolist(), std=std.tolist())

    if 'Denoise' in model_name:
        net = nn.Sequential(
            # Normalize(mean, std),
            normalize,
            Permute([2,1,0]),
            pretrained_model
        )

    else:
        net = nn.Sequential(
            # Normalize(mean, std),
            normalize,
            pretrained_model
        )


    if torch.cuda.is_available():
        net = net.cuda()
    net.eval()
    return net

Overwriting /content/BlackboxBench/utils/model_loader.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/decision/sign_opt_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import time
import scipy.spatial
from scipy.linalg import qr
import random
from torch import Tensor as t

from attacks.decision.decision_black_box_attack import DecisionBlackBoxAttack

start_learning_rate = 1.0

def quad_solver(Q, b):
    """
    Solve min_a  0.5*aQa + b^T a s.t. a>=0
    """
    K = Q.shape[0]
    alpha = torch.zeros((K,))
    g = b
    Qdiag = torch.diag(Q)
    for _ in range(20000):
        delta = torch.maximum(alpha - g/Qdiag,0) - alpha
        idx = torch.argmax(torch.abs(delta))
        val = delta[idx]
        if abs(val) < 1e-7:
            break
        g = g + val*Q[:,idx]
        alpha[idx] += val
    return alpha

def sign(y):
    """
    y -- numpy array of shape (m,)
    Returns an element-wise indication of the sign of a number.
    The sign function returns -1 if y < 0, 1 if x >= 0. nan is returned for nan inputs.
    """
    y_sign = torch.sign(y)
    y_sign[y_sign==0] = 1
    return y_sign


class SignOPTAttack(DecisionBlackBoxAttack):
    """
    Sign_OPT
    """

    def __init__(self, epsilon, p, alpha, beta, svm, momentum, max_queries, k, lb, ub, batch_size, sigma):
        super().__init__(max_queries = max_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size = batch_size)
        self.alpha = alpha
        self.beta = beta
        self.svm = svm
        self.momentum = momentum
        self.k = k
        self.sigma = sigma
        self.query_count = 0


    def _config(self):
        return {
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "attack_name": self.__class__.__name__
        }

    def attack_untargeted(self, x0, y0, alpha = 0.2, beta = 0.001):
        """
        Attack the original image and return adversarial example
        """

        y0 = y0[0]
        self.query_count = 0

        # Calculate a good starting point.
        num_directions = 10
        best_theta, g_theta = None, float('inf')
        print("Searching for the initial direction on %d random directions: " % (num_directions))
        timestart = time.time()
        for i in range(num_directions):
            self.query_count += 1
            theta = torch.randn_like(x0)
            if self.predict_label(x0+theta)!=y0:
                initial_lbd = torch.norm(theta)
                theta /= initial_lbd
                lbd, count = self.fine_grained_binary_search(x0, y0, theta, initial_lbd, g_theta)
                self.query_count += count
                if lbd < g_theta:
                    best_theta, g_theta = theta, lbd
                    # print("--------> Found distortion %.4f" % g_theta)

        timeend = time.time()
        if g_theta == float('inf'):
            print("Couldn't find valid initial, failed")
            return x0, self.query_count
        # print("==========> Found best distortion %.4f in %.4f seconds "
        #       "using %d queries" % (g_theta, timeend-timestart, self.query_count))

        # Begin Gradient Descent.
        timestart = time.time()
        xg, gg = best_theta, g_theta
        vg = torch.zeros_like(xg)

        for i in range(1500):
            if self.svm == True:
                sign_gradient, grad_queries = self.sign_grad_svm(x0, y0, xg, initial_lbd=gg, h=beta)
            else:
                sign_gradient, grad_queries = self.sign_grad_v1(x0, y0, xg, initial_lbd=gg, h=beta)
            self.query_count += grad_queries
            # Line search
            min_theta = xg
            min_g2 = gg
            min_vg = vg
            for _ in range(15):
                if self.momentum > 0:
                    new_vg = self.momentum*vg - alpha*sign_gradient
                    new_theta = xg + new_vg
                else:
                    new_theta = xg - alpha * sign_gradient
                new_theta /= torch.norm(new_theta)
                new_g2, count = self.fine_grained_binary_search_local(x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                self.query_count += count
                alpha = alpha * 2
                if new_g2 < min_g2:
                    min_theta = new_theta
                    min_g2 = new_g2
                    if self.momentum > 0:
                        min_vg = new_vg
                else:
                    break
            if min_g2 >= gg:
                for _ in range(15):
                    alpha = alpha * 0.25
                    if self.momentum > 0:
                        new_vg = self.momentum*vg - alpha*sign_gradient
                        new_theta = xg + new_vg
                    else:
                        new_theta = xg - alpha * sign_gradient
                    new_theta /= torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local(x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                    self.query_count += count
                    if new_g2 < gg:
                        min_theta = new_theta
                        min_g2 = new_g2
                        if self.momentum > 0:
                            min_vg = new_vg
                        break
            if alpha < 1e-4:
                alpha = 1.0
                print("Warning: not moving")
                beta = beta*0.1
                if (beta < 1e-8):
                    break

            xg, gg = min_theta, min_g2
            vg = min_vg


            if self.query_count > self.max_queries:
               break

            dist = self.distance(gg*xg)
            if dist < self.epsilon:
                break

            # print("Iteration %3d distortion %.4f num_queries %d" % (i+1, dist, self.query_count))

        dist = self.distance(gg*xg)
        timeend = time.time()
        # print("\nAdversarial Example Found Successfully: distortion %.4f queries %d \nTime: %.4f seconds" % (dist, self.query_count, timeend-timestart))
        return x0 + gg*xg, self.query_count

    def sign_grad_v1(self, x0, y0, theta, initial_lbd, h=0.001, D=4, target=None):
        """
        Evaluate the sign of gradient by formulat
        sign(g) = 1/Q [ \sum_{q=1}^Q sign( g(theta+h*u_i) - g(theta) )u_i$ ]
        """
        K = self.k
        sign_grad = torch.zeros_like(theta)
        queries = 0
        for _ in range(K):
            u = torch.randn_like(theta)
            u /= torch.norm(u)

            sign = 1
            new_theta = theta + h*u
            new_theta /= torch.norm(new_theta)

            # Targeted case.
            if (target is not None and
                self.predict_label(x0+initial_lbd*new_theta) == target):
                sign = -1

            # Untargeted case
            if (target is None and
                self.predict_label(x0+t(initial_lbd*new_theta)) != y0):
                sign = -1
            queries += 1
            sign_grad += u*sign

        sign_grad /= K

        return sign_grad, queries

    def sign_grad_svm(self, x0, y0, theta, initial_lbd, h=0.001, K=100, lr=5.0, target=None):
        """
        Evaluate the sign of gradient by formulat
        sign(g) = 1/Q [ \sum_{q=1}^Q sign( g(theta+h*u_i) - g(theta) )u_i$ ]
        """
        sign_grad = torch.zeros_like(theta)
        queries = 0
        dim = np.prod(theta.shape)
        X = torch.zeros((dim, K))
        for iii in range(K):
            u = torch.randn_like(theta)
            u /= torch.norm(u)

            sign = 1
            new_theta = theta + h*u
            new_theta /= torch.norm(new_theta)

            # Targeted case.
            if (target is not None and
                self.predict_label(x0+t(initial_lbd*new_theta)) == target):
                sign = -1

            # Untargeted case
            if (target is None and
                self.predict_label(x0+t(initial_lbd*new_theta)) != y0):
                sign = -1

            queries += 1
            X[:,iii] = sign*u.reshape((dim,))

        Q = X.transpose().dot(X)
        q = -1*torch.ones((K,))
        # G = torch.diag(-1*torch.ones((K,)))
        # h = torch.zeros((K,))
        ### Use quad_qp solver
        #alpha = solve_qp(Q, q, G, h)
        ### Use coordinate descent solver written by myself, avoid non-positive definite cases
        alpha = quad_solver(Q, q)
        sign_grad = (X.dot(alpha)).reshape(theta.shape)

        return sign_grad, queries

    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd

        if self.predict_label(x0+lbd*theta) == y0:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            while self.predict_label(x0+lbd_hi*theta) == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 20:
                    return float('inf'), nquery
        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
            while self.predict_label(x0+lbd_lo*theta) != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1
                if nquery + self.query_count> self.max_queries:
                    break

        while (lbd_hi - lbd_lo) > tol:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if nquery + self.query_count> self.max_queries:
                break
            if self.predict_label(x0 + lbd_mid*theta) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery

    def fine_grained_binary_search(self, x0, y0, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best:
            if self.predict_label(x0+t(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd

        lbd_hi = lbd
        lbd_lo = 0.0

        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if nquery + self.query_count> self.max_queries:
                break
            if self.predict_label(x0 + t(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery

    def attack_targeted(self, x0, target, alpha = 0.2, beta = 0.001):
        """
        Attack the original image and return adversarial example
        """

        target = target[0]

        num_samples = 100
        best_theta, g_theta = None, float('inf')
        query_count = 0
        ls_total = 0
        sample_count = 0
        print("Searching for the initial direction on %d samples: " % (num_samples))
        timestart = time.time()


        # Iterate through training dataset. Find best initial point for gradient descent.
        for i in range(500):
            xi, _ = self.train_dataset.get_eval_data(i,i+1)
            yi_pred = self.predict_label(xi)
            query_count += 1
            if yi_pred != target:
                continue
            theta = xi - x0
            initial_lbd = torch.norm(theta)
            theta /= initial_lbd
            lbd, count = self.fine_grained_binary_search_targeted(x0, target, theta, initial_lbd, g_theta)
            query_count += count
            if lbd < g_theta:
                best_theta, g_theta = theta, lbd
                # print("--------> Found distortion %.4f" % g_theta)

            sample_count += 1
            if sample_count >= num_samples:
                break


        timeend = time.time()
        if g_theta == np.inf:
            return x0, query_count
        # print("==========> Found best distortion %.4f in %.4f seconds using %d queries" %
        #       (g_theta, timeend-timestart, query_count))

        # Begin Gradient Descent.
        timestart = time.time()
        xg, gg = best_theta, g_theta

        for i in range(1000):
            if self.svm == True:
                sign_gradient, grad_queries = self.sign_grad_svm(x0, 0, xg, initial_lbd=gg, h=beta, target=target)
            else:
                sign_gradient, grad_queries = self.sign_grad_v1(x0, 0, xg, initial_lbd=gg, h=beta, target=target)


            # Line search
            ls_count = 0
            min_theta = xg
            min_g2 = gg
            for _ in range(15):
                new_theta = xg - alpha * sign_gradient
                new_theta /= torch.norm(new_theta)
                new_g2, count = self.fine_grained_binary_search_local_targeted(x0, target, new_theta, initial_lbd = min_g2, tol=beta/500)
                ls_count += count
                alpha = alpha * 2
                if new_g2 < min_g2:
                    min_theta = new_theta
                    min_g2 = new_g2
                else:
                    break

            if min_g2 >= gg:
                for _ in range(15):
                    alpha = alpha * 0.25
                    new_theta = xg - alpha * sign_gradient
                    new_theta /= torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local_targeted(x0, target, new_theta, initial_lbd = min_g2, tol=beta/500)
                    ls_count += count
                    if new_g2 < gg:
                        min_theta = new_theta
                        min_g2 = new_g2
                        break

            if alpha < 1e-4:
                alpha = 1.0
                print("Warning: not moving")
                beta = beta*0.1
                if (beta < 1e-8):
                    break

            xg, gg = min_theta, min_g2

            query_count += (grad_queries + ls_count)
            ls_total += ls_count

            if query_count > self.max_queries:
                break

            dist = self.distance(gg*xg)
            if dist < self.epsilon:
                break

            if i%5==0:
                print("Iteration %3d distortion %.4f num_queries %d" % (i+1, dist, query_count))

        adv_target = self.predict_label(x0 + t(gg*xg))
        if (adv_target == target):
            timeend = time.time()
            print("\nAdversarial Example Found Successfully: distortion %.4f target"
                  " %d queries %d LS queries %d \nTime: %.4f seconds" % (dist, target, query_count, ls_total, timeend-timestart))

            return x0 + t(gg*xg), query_count
        else:
            print("Failed to find targeted adversarial example.")
            return x0, query_count

    def fine_grained_binary_search_local_targeted(self, x0, t, theta, initial_lbd=1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd

        if self.predict_label(x0 + t(lbd*theta)) != t:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            while self.predict_label(x0 + t(lbd_hi*theta)) != t:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 100:
                    return float('inf'), nquery
        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
            while self.predict_label(x0 + t(lbd_lo*theta)) == t:
                lbd_lo = lbd_lo*0.99
                nquery += 1

        while (lbd_hi - lbd_lo) > tol:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.predict_label(x0 + t(lbd_mid*theta)) == t:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid

        return lbd_hi, nquery

    def fine_grained_binary_search_targeted(self, x0, t, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best:
            if self.predict_label(x0 + t(current_best*theta)) != t:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd

        lbd_hi = lbd
        lbd_lo = 0.0

        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.predict_label(x0 + t(lbd_mid*theta)) != t:
                lbd_lo = lbd_mid
            else:
                lbd_hi = lbd_mid
        return lbd_hi, nquery


    def _perturb(self, xs_t, ys):
        if self.targeted:
            adv, q = self.attack_targeted(xs_t, ys, self.alpha, self.beta)
        else:
            adv, q = self.attack_untargeted(xs_t, ys, self.alpha, self.beta)

        return adv, q

'''

Original License

MIT License

Copyright (c) 2022 Minhao Cheng
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/decision/sign_opt_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/decision/hsja_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
from torch import Tensor as t

from attacks.decision.decision_black_box_attack import DecisionBlackBoxAttack


class HSJAttack(DecisionBlackBoxAttack):
    """
    HSJA
    """
    def __init__(self, epsilon, p, max_queries, gamma, stepsize_search, max_num_evals, init_num_evals, EOT, sigma, lb, ub, batch_size):
        super().__init__(max_queries = max_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size = batch_size)
        self.gamma = gamma
        self.stepsize_search = stepsize_search
        self.max_num_evals = max_num_evals
        self.init_num_evals = init_num_evals
        self.verbose = True
        self.EOT = EOT
        self.sigma = sigma

    def _config(self):
        return {
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "attack_name": self.__class__.__name__
        }

    def hsja(self,input_xi,label_or_target):

        d = int(np.prod(input_xi.shape))
        # Set binary search threshold.
        if self.p == '2':
                theta = self.gamma / (np.sqrt(d) * d)
        else:
                theta = self.gamma / (d ** 2)

        self.query = 0
        # Initialize.
        perturbed = self.initialize(input_xi, label_or_target)


        # Project the initialization to the boundary.
        perturbed, dist_post_update = self.binary_search_batch(input_xi, perturbed, label_or_target, theta)
        dist = self.compute_distance(perturbed, input_xi)

        for j in np.arange(10000):
                # Choose delta.
                if j==1:
                    delta = 0.1 * (self.ub - self.lb)
                else:
                    if self.p == '2':
                            delta = np.sqrt(d) * theta * dist_post_update
                    elif self.p == 'inf':
                            delta = d * theta * dist_post_update

                # Choose number of evaluations.
                num_evals = int(self.init_num_evals * np.sqrt(j+1))
                num_evals = int(min([num_evals, self.max_num_evals]))

                # approximate gradient.
                gradf = self.approximate_gradient(perturbed, label_or_target, num_evals, delta)
                if self.p == 'inf':
                        update = np.sign(gradf)
                else:
                        update = gradf

                # search step size.
                if self.stepsize_search == 'geometric_progression':
                        # find step size.
                        epsilon = self.geometric_progression_for_stepsize(perturbed, label_or_target,
                                update, dist, j+1)

                        # Update the sample.
                        perturbed = self.clip_image(perturbed + epsilon * update,
                                self.lb, self.ub)

                        # Binary search to return to the boundary.
                        perturbed, dist_post_update = self.binary_search_batch(input_xi,
                                perturbed[None], label_or_target, theta)

                elif self.stepsize_search == 'grid_search':
                        # Grid search for stepsize.
                        epsilons = np.logspace(-4, 0, num=20, endpoint = True) * dist
                        epsilons_shape = [20] + len(input_xi.shape) * [1]
                        perturbeds = perturbed + epsilons.reshape(epsilons_shape) * update
                        perturbeds = self.clip_image(perturbeds, self.lb, self.ub)
                        idx_perturbed = self.decision_function(perturbeds, label_or_target)

                        if np.sum(idx_perturbed) > 0:
                                # Select the perturbation that yields the minimum distance # after binary search.
                                perturbed, dist_post_update = self.binary_search_batch(input_xi,
                                        perturbeds[idx_perturbed], label_or_target, theta)

                # compute new distance.
                dist = self.compute_distance(perturbed, input_xi)
                if self.verbose:
                        print('iteration: {:d}, {:s} distance {:.4f}'.format(j+1, self.p, dist))
                if self.query > self.max_queries:
                        break
                if dist < self.epsilon:
                        break

        return t(perturbed).unsqueeze(0)

    def decision_function(self, images, label):
        """
        Decision function output 1 on the desired side of the boundary,
        0 otherwise.
        """
        images = torch.from_numpy(images).float()
        if len(images.shape) == 3:
                images = images.unsqueeze(0)
        self.query += images.shape[0]
        la = self.predict_label(images).cpu().numpy()
        if self.targeted:
                return (la==label)
        else:
                return (la!=label)

    def clip_image(self, image, clip_min, clip_max):
        # Clip an image, or an image batch, with upper and lower threshold.
        return np.minimum(np.maximum(clip_min, image), clip_max)


    def compute_distance(self, x_ori, x_pert):
        # Compute the distance between two images.
        if self.p == '2':
                return np.linalg.norm(x_ori - x_pert)
        elif self.p == 'inf':
                return np.max(abs(x_ori - x_pert))


    def approximate_gradient(self, sample, label_or_target, num_evals, delta):
        # Generate random vectors.
        noise_shape = [num_evals] + list(sample.shape)
        if self.p == '2':
                rv = np.random.randn(*noise_shape)
        elif self.p == 'inf':
                rv = np.random.uniform(low = -1, high = 1, size = noise_shape)

        rv = rv / np.sqrt(np.sum(rv ** 2, axis = (1,2,3), keepdims = True))
        perturbed = sample + delta * rv
        perturbed = self.clip_image(perturbed, self.lb, self.ub)
        rv = (perturbed - sample) / delta

        # query the model.
        decisions = self.decision_function(perturbed, label_or_target)
        decision_shape = [len(decisions)] + [1] * len(sample.shape)
        fval = 2 * decisions.astype(float).reshape(decision_shape) - 1.0

        # Baseline subtraction (when fval differs)
        if np.mean(fval) == 1.0: # label changes.
                gradf = np.mean(rv, axis = 0)
        elif np.mean(fval) == -1.0: # label not change.
                gradf = - np.mean(rv, axis = 0)
        else:
                fval -= np.mean(fval)
                gradf = np.mean(fval * rv, axis = 0)

        # Get the gradient direction.
        gradf = gradf / np.linalg.norm(gradf)

        return gradf


    def project(self, original_image, perturbed_images, alphas):
        alphas_shape = [1] * len(original_image.shape)
        alphas = alphas.reshape(alphas_shape)
        if self.p == '2':
                #print(alphas.shape,original_image.shape, perturbed_images.shape)
                return (1-alphas) * original_image + alphas * perturbed_images
        elif self.p == 'inf':
                out_images = self.clip_image(
                        perturbed_images,
                        original_image - alphas,
                        original_image + alphas
                        )
                return out_images


    def binary_search_batch(self, original_image, perturbed_images, label_or_target, theta):
        """ Binary search to approach the boundary. """

        # Compute distance between each of perturbed image and original image.
        dists_post_update = np.array([
                        self.compute_distance(
                                original_image,
                                perturbed_image
                        )
                        for perturbed_image in perturbed_images])
        #print(dists_post_update)
        # Choose upper thresholds in binary searchs based on constraint.
        if self.p == 'inf':
                highs = dists_post_update
                # Stopping criteria.
                thresholds = np.minimum(dists_post_update * theta, theta)
        else:
                highs = np.ones(len(perturbed_images))
                thresholds = theta

        lows = np.zeros(len(perturbed_images))



        # Call recursive function.
        while np.max((highs - lows) / thresholds) > 1:
                # projection to mids.
                mids = (highs + lows) / 2.0
                mid_images = self.project(original_image, perturbed_images, mids)
                # Update highs and lows based on model decisions.
                decisions = self.decision_function(mid_images, label_or_target)
                lows = np.where(decisions == 0, mids, lows)
                highs = np.where(decisions == 1, mids, highs)
                if self.query > self.max_queries:
                        break

        out_images = self.project(original_image, perturbed_images, highs)

        # Compute distance of the output image to select the best choice.
        # (only used when stepsize_search is grid_search.)
        dists = np.array([
                self.compute_distance(
                        original_image,
                        out_image
                )
                for out_image in out_images])
        idx = np.argmin(dists)

        dist = dists_post_update[idx]
        out_image = out_images[idx]
        return out_image, dist


    def initialize(self, input_xi, label_or_target):
        """
        Efficient Implementation of BlendedUniformNoiseAttack in Foolbox.
        """
        success = 0
        num_evals = 0
        # Find a misclassified random noise.
        while True:
                random_noise = np.random.uniform(self.lb, self.ub, size = input_xi.shape)
                success = self.decision_function(random_noise, label_or_target)[0]
                if success:
                        break
                if self.query > self.max_queries:
                        break
                assert num_evals < 1e4,"Initialization failed! "
                "Use a misclassified image as `target_image`"

        # Binary search to minimize l2 distance to original image.
        low = 0.0
        high = 1.0
        while high - low > 0.001:
                mid = (high + low) / 2.0
                blended = (1 - mid) * input_xi + mid * random_noise
                success = self.decision_function(blended, label_or_target)
                if success:
                        high = mid
                else:
                        low = mid
                if self.query > self.max_queries:
                        break

        initialization = (1 - high) * input_xi + high * random_noise

        return initialization


    def geometric_progression_for_stepsize(self, x, label_or_target, update, dist, j):
        """
        Geometric progression to search for stepsize.
        Keep decreasing stepsize by half until reaching
        the desired side of the boundary,
        """
        epsilon = dist / np.sqrt(j)

        def phi(epsilon):
                new = x + epsilon * update
                success = self.decision_function(new, label_or_target)
                return success

        while not phi(epsilon):
                epsilon /= 2.0
                if self.query > self.max_queries:
                        break

        return epsilon


    def _perturb(self, xs_t, ys_t):
        xs = xs_t.cpu().numpy()
        ys = ys_t.cpu().numpy()
        adv = self.hsja(xs, ys)
        return t(adv), self.query

'''

Original License

MIT License

Copyright (c) 2022 Minhao Cheng

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Overwriting /content/BlackboxBench/attacks/decision/hsja_attack.py


In [ ]:
%%writefile /content/BlackboxBench/attacks/decision/geoda_attack.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import os
import time
from torch import Tensor as t
from numpy import linalg
import math

from attacks.decision.decision_black_box_attack import DecisionBlackBoxAttack

from math import cos, sqrt, pi


def dct(x, y, v, u, n):
    # Normalisation
    def alpha(a):
        if a == 0:
            return sqrt(1.0 / n)
        else:
            return sqrt(2.0 / n)

    return alpha(u) * alpha(v) * cos(((2 * x + 1) * (u * pi)) / (2 * n)) * cos(((2 * y + 1) * (v * pi)) / (2 * n))


def generate_2d_dct_basis(sub_dim, n, path):
    # Assume square image, so we don't have different xres and yres

    # We can get different frequencies by setting u and v
    # Here, we have a max u and v to loop over and display
    # Feel free to adjust
    maxU = sub_dim
    maxV = sub_dim

    dct_basis = []
    for u in range(0, maxU):
        for v in range(0, maxV):
            basisImg = np.zeros((n, n))
            for y in range(0, n):
                for x in range(0, n):
                    basisImg[y, x] = dct(x, y, v, u, max(n, maxV))
            dct_basis.append(basisImg)
    dct_basis = np.mat(np.reshape(dct_basis, (maxV*maxU, n*n))).transpose()
    np.save(path, dct_basis)
    return dct_basis

class SubNoise(torch.nn.Module):
    """given subspace x and the number of noises, generate sub noises"""
    # x is the subspace basis
    def __init__(self, num_noises, x):
        self.num_noises = num_noises
        self.x = x
        self.size = int(self.x.shape[0] ** 0.5)
        super(SubNoise, self).__init__()

    def forward(self):
        r = torch.zeros([self.size ** 2, 3*self.num_noises], dtype=torch.float32)
        noise = torch.randn([self.x.shape[1], 3*self.num_noises], dtype=torch.float32).cuda()
        sub_noise = torch.transpose(torch.mm(self.x, noise), 0, 1)
        r = sub_noise.view([self.num_noises, 3, self.size, self.size])
        r_list = r.permute(0,2,3,1)
        return r_list

class GeoDAttack(DecisionBlackBoxAttack):
    """
    GeoDA
    """
    def __init__(self, epsilon, p, max_queries, sub_dim, tol, alpha, mu, search_space, grad_estimator_batch_size, lb, ub, batch_size, sigma):
        super().__init__(max_queries = max_queries,
                         epsilon=epsilon,
                         p=p,
                         lb=lb,
                         ub=ub,
                         batch_size = batch_size)
        self.sub_dim = sub_dim
        self.tol = tol
        self.alpha = alpha
        self.mu = mu
        self.search_space = search_space
        self.grad_estimator_batch_size = grad_estimator_batch_size
        self.sigma = sigma

    def _config(self):
        return {
            "p": self.p,
            "epsilon": self.epsilon,
            "lb": self.lb,
            "ub": self.ub,
            "attack_name": self.__class__.__name__
        }

    def opt_query_iteration(self, Nq, T, eta):
        coefs=[eta**(-2*i/3) for i in range(0,T)]
        coefs[0] = 1*coefs[0]
        sum_coefs = sum(coefs)
        opt_q=[round(Nq*coefs[i]/sum_coefs) for i in range(0,T)]
        if opt_q[0]>80:
            T = T + 1
            opt_q, T = self.opt_query_iteration(Nq, T, eta)
        elif opt_q[0]<50:
            T = T - 1
            opt_q, T = self.opt_query_iteration(Nq, T, eta)

        return opt_q, T

    def find_random_adversarial(self, x0, y0, epsilon=1000):

        num_calls = 1

        step = 0.02
        perturbed = x0

        while self.predict_label(perturbed) == y0[0]:
            pert = torch.randn(x0.shape)

            perturbed = x0 + num_calls*step* pert
            perturbed = torch.clamp(perturbed, 0, 1)
            num_calls += 1

        return perturbed, num_calls

    def bin_search(self, x_0, y_0, x_random, tol):
        num_calls = 0
        adv = x_random
        cln = x_0
        while True:

            mid = (cln + adv) / 2.0
            num_calls += 1

            if self.predict_label(mid) != y_0[0]:
                adv = mid
            else:
                cln = mid

            if torch.norm(adv-cln)<tol:
                break

        return adv, num_calls

    def go_to_boundary(self, x_0, y_0, grad):

        epsilon = 1

        num_calls = 1
        perturbed = x_0

        if self.p == '2':
            grads = grad
        elif self.p == 'inf':
            grads = torch.sign(grad)/torch.norm(grad)

        while self.predict_label(perturbed) == y_0[0]:
            perturbed = x_0 + (num_calls*epsilon*grads[0])
            perturbed = torch.clamp(perturbed, 0, 1)

            num_calls += 1

            if num_calls > 100:
                print('falied ... ')
                break
        return perturbed, num_calls

    def black_grad_batch(self, x_boundary, q_max, random_noises, y_0, sub_basis_torch):
        grad_tmp = [] # estimated gradients in each estimate_batch
        z = []        # sign of grad_tmp
        outs = []
        batch_size = self.grad_estimator_batch_size

        num_batchs = math.ceil(q_max/batch_size)
        last_batch = q_max - (num_batchs-1)*batch_size
        EstNoise = SubNoise(batch_size, sub_basis_torch)
        all_noises = []

        for j in range(num_batchs):
            if j == num_batchs-1:
                EstNoise_last = SubNoise(last_batch, sub_basis_torch)
                current_batch = EstNoise_last()
                current_batch_np = current_batch.cpu().numpy()
                noisy_boundary = [x_boundary[0,:,:,:].cpu().numpy()]*last_batch +self.alpha*current_batch.cpu().numpy()

            else:
                current_batch = EstNoise()
                current_batch_np = current_batch.cpu().numpy()
                noisy_boundary = [x_boundary[0,:,:,:].cpu().numpy()]*batch_size +self.alpha*current_batch.cpu().numpy()

            all_noises.append(current_batch_np)

            noisy_boundary_tensor = torch.tensor(noisy_boundary)

            predict_labels = self.predict_label(noisy_boundary_tensor)

            outs.append(predict_labels.cpu())

        all_noise = np.concatenate(all_noises, axis=0)
        outs = np.concatenate(outs, axis=0)


        for i, predict_label in enumerate(outs):
            if predict_label == y_0:
                z.append(1)
                grad_tmp.append(all_noise[i])
            else:
                z.append(-1)
                grad_tmp.append(-all_noise[i])

        grad = -(1/q_max)*sum(grad_tmp)

        grad_f = torch.tensor(grad)[None, :,:,:]

        return grad_f, sum(z)



    def GeoDA(self, x_0, y_0, x_b, iteration, q_opt, sub_basis_torch):
        q_num = 0
        grad = 0

        for i in range(iteration):

            t1 = time.time()
            random_vec_o = torch.randn(q_opt[i], x_0.shape[1], x_0.shape[2], x_0.shape[3])

            grad_oi, _ = self.black_grad_batch(x_b, q_opt[i], random_vec_o, y_0[0], sub_basis_torch)
            q_num = q_num + q_opt[i]

            grad = grad_oi + grad
            x_adv, qs = self.go_to_boundary(x_0, y_0, grad)
            q_num = q_num + qs

            x_adv, bin_query = self.bin_search(x_0, y_0, x_adv, self.tol)
            q_num = q_num + bin_query

            x_b = x_adv

            t2 = time.time()

            norm = self.distance(x_adv, x_0)
            message = ' (took {:.5f} seconds)'.format(t2 - t1)
            print('iteration -> ' + str(i) + str(message) + '     -- ' + self.p + ' norm is -> ' + str(norm))
            if norm < self.epsilon:
                break
            if q_num > self.max_queries:
                break


        x_adv = torch.clamp(x_adv, 0, 1)

        return x_adv, q_num, grad

    def _perturb(self, xs_t, ys):

        if self.search_space == 'sub':
            print('Check if DCT basis available ...')

            path = os.path.join(os.path.dirname(__file__), '2d_dct_basis_{}_{}.npy'.format(self.sub_dim,xs_t.size(2)))
            if os.path.exists(path):
                print('Yes, we already have it ...')
                sub_basis = np.load(path).astype(np.float32)
            else:
                print('Generating dct basis ......')
                sub_basis = generate_2d_dct_basis(self.sub_dim, xs_t.size(2), path).astype(np.float32)
                print('Done!\n')


            sub_basis_torch = torch.from_numpy(sub_basis).cuda()


        x_random, query_random_1 = self.find_random_adversarial(xs_t, ys, epsilon=100)

        # Binary search

        x_boundary, query_binsearch_2 = self.bin_search(xs_t, ys, x_random, self.tol)
        x_b = x_boundary


        query_rnd = query_binsearch_2 + query_random_1


        iteration = round(self.max_queries/500)
        q_opt_it = int(self.max_queries  - (iteration)*25)
        q_opt_iter, iterate = self.opt_query_iteration(q_opt_it, iteration, self.mu)
        q_opt_it = int(self.max_queries  - (iterate)*25)
        q_opt_iter, iterate = self.opt_query_iteration(q_opt_it, iteration, self.mu)
        print('Start: The GeoDA will be run for:' + ' Iterations = ' + str(iterate) + ', Query = ' + str(self.max_queries) + ', Norm = ' + str(self.p)+ ', Space = ' + str(self.search_space) )


        t3 = time.time()
        adv, query_o, _= self.GeoDA(xs_t, ys, x_b, iterate, q_opt_iter, sub_basis_torch)
        t4 = time.time()
        message = ' took {:.5f} seconds'.format(t4 - t3)
        qmessage = ' with query = ' + str(query_o + query_rnd)


        print('End: The GeoDA algorithm' + message + qmessage )


        return adv, query_o + query_rnd


Overwriting /content/BlackboxBench/attacks/decision/geoda_attack.py


In [ ]:
%%writefile /content/BlackboxBench/BaselinesExperiment.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import math
import os
import time
import sys
sys.path.append(os.path.join(os.path.dirname(os.path.realpath(__file__)), ".."))

import numpy as np
import pandas as pd
import tensorflow as tf
# import torch as ch
import torch
from advertorch.defenses import *
from datasets.dataset import Dataset
from utils.compute import tf_nsign, sign
from utils.misc import config_path_join, src_path_join, create_dir, get_dataset_shape
from utils.model_loader import load_torch_models, load_torch_models_imagesub
from utils.compute import tf_nsign, sign, linf_proj_maker, l2_proj_maker
from art.defences.preprocessor import Preprocessor, GaussianAugmentation, FeatureSqueezing
import torch.nn as nn
from attacks.score.bandit_attack import BanditAttack
from attacks.score.zo_sign_sgd_attack import ZOSignSGDAttack
from attacks.score.sign_attack import SignAttack
from attacks.score.simple_attack import SimpleAttack
from attacks.score.square_attack import SquareAttack
from attacks.score.parsimonious_attack import ParsimoniousAttack
from attacks.score.nes_attack import NESAttack
from attacks.decision.sign_opt_attack import SignOPTAttack
from attacks.decision.hsja_attack import HSJAttack
from attacks.decision.geoda_attack import GeoDAttack
import ast

def Counter_Samples(iters,model, gaus, x_preprocessed, k):
    import torch
    # Apply noise first
    use_cuda = True
    device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
    x_preprocessed = torch.from_numpy(gaus(x_preprocessed.detach().cpu().numpy())[0]).to(device)
    x_preprocessed.requires_grad_(True)
    x_preprocessed.retain_grad()
    loss = nn.CrossEntropyLoss(reduction='none')
    for iter in range(iters):
        # predicting labels
        model_output = model(x_preprocessed)
        true_labels_indexes = torch.argmax(model_output, dim=1)
        loss_comp = loss(model_output, true_labels_indexes)
        loss_comp.backward(torch.ones_like(loss_comp))
        # update the samples.
        x_preprocessed = x_preprocessed - k * x_preprocessed.grad # No normalization.
        x_preprocessed.retain_grad()
    return x_preprocessed


if __name__ == '__main__':

	print("Running Baselines Experiment..")
	version_dict = {
        'RND': False,
        'CounterSamples_10': False,
        'Counter_Samples_1': False,
        'SND': False,
        'FS': False,
        'BS': False,
        'AS': False,
        'JPEG': False
    }
	device = 'cuda' if torch.cuda.is_available() else 'cpu'
	baselines_json = os.sys.argv[2]
	data_received = json.loads(baselines_json)

	baselines = data_received["Baselines"]
	print(baselines)
	for version in baselines:
		version_dict[version] = True

		# Defense Selection
		FG = version_dict['Counter_Samples_1']
		PGD_FG = version_dict['CounterSamples_10']
		RND = version_dict['RND']
		gaussian_noise = version_dict['SND']
		feature_squeezing = version_dict['FS']
		bit_squeezing = version_dict['BS']
		jpeg_compression = version_dict['JPEG']
		average_smoothing = version_dict['AS']


		gaus = GaussianAugmentation(augmentation=False, sigma=0.01)
		bs = BitSqueezing(bit_depth=3)
		jc = JPEGFilter(quality=75)
		cs = AverageSmoothing2D(kernel_size=3, channels=3).to(device)
		fs = FeatureSqueezing(clip_values=(0,1),bit_depth = 3)
		config = os.sys.argv[1]
		exp_id = config.split('/')[-1]
		res = {}
		cfs = [config]
		for _cf in cfs:
			config_file = config_path_join(_cf)

			with open(config_file) as config_file:
				config = json.load(config_file)

			pgd_iters = config['defence_config']['k']
			step_size_k = config['defence_config']['alpha']

			# for reproducibility
			nname = config['attack_name']
			print(f'Now running the experiment on :\n Attack: {nname} \n Defence :{version}')
			data_dir = f'Results/{nname}/{version}'
			create_dir(data_dir)
			print(f'creating folder {data_dir}')
			seed = config['seed']
			np.random.seed(seed)
			torch.manual_seed(seed)
			torch.cuda.manual_seed(seed)

			dset = Dataset(config['dset_name'], config)

			model_name = config['modeln']
			if config['dset_name'] == 'imagenet':
				model = load_torch_models_imagesub(model_name)
			else:
				model = load_torch_models(model_name)

			print('The Black-Box model: {}'.format(config['modeln']))


			p_norm = config['attack_config']['p']
			print("The attack norm constrain is: {} norm".format(p_norm))
			epsilon = config['attack_config']['epsilon'] / 255.

			# set torch default device:
			if torch.cuda.is_available():
				torch.set_default_tensor_type('torch.cuda.FloatTensor')
			else:
				torch.set_default_tensor_type('torch.FloatTensor')

			def cw_loss(logit, label, target=False):
				if target:
					# targeted cw loss: logit_t - max_{i\neq t}logit_i
					_, argsort = logit.sort(dim=1, descending=True)
					target_is_max = argsort[:, 0].eq(label)
					second_max_index = target_is_max.long() * argsort[:, 1] + (~ target_is_max).long() * argsort[:, 0]
					target_logit = logit[torch.arange(logit.shape[0]), label]
					second_max_logit = logit[torch.arange(logit.shape[0]), second_max_index]
					return target_logit - second_max_logit
				else:
					# untargeted cw loss: max_{i\neq y}logit_i - logit_y
					_, argsort = logit.sort(dim=1, descending=True)
					gt_is_max = argsort[:, 0].eq(label)
					second_max_index = gt_is_max.long() * argsort[:, 1] + (~gt_is_max).long() * argsort[:, 0]
					gt_logit = logit[torch.arange(logit.shape[0]), label]
					second_max_logit = logit[torch.arange(logit.shape[0]), second_max_index]
					return second_max_logit - gt_logit

			def xent_loss(logit, label, target=False):
				if not target:
					return torch.nn.CrossEntropyLoss(reduction='none')(logit, label)
				else:
					return -torch.nn.CrossEntropyLoss(reduction='none')(logit, label)

			# criterion = torch.nn.CrossEntropyLoss(reduce=False)
			# criterion = xent_loss
			criterion = cw_loss


			attacker = eval(config['attack_name'])(
				**config['attack_config'],
				lb=dset.min_value,
				ub=dset.max_value
			)

			print(attacker._config())

			target = config["target"]


			# Iterate over the samples batch-by-batch
			num_eval_examples = config['num_eval_examples']
			eval_batch_size = config['attack_config']['batch_size']
			num_batches = int(math.ceil(num_eval_examples / eval_batch_size))


			print('Iterating over {} batches'.format(num_batches))
			start_time = time.time()
			def activate_defense(model, data, k=step_size_k, pgd_iters=pgd_iters):
				# k = step_size_k
				if RND:
					sigma = 0.02
					data = data + sigma * torch.randn_like(data)
				if PGD_FG:
					data = Counter_Samples(pgd_iters, model, gaus, data, k)
				if FG:
					data = Counter_Samples(1, model, gaus, data, k)
				if gaussian_noise:
					data = torch.from_numpy(gaus(data.detach().cpu().numpy())[0]).to(device)
				if feature_squeezing:
					data = torch.tensor(fs(data.detach().cpu().numpy())[0]).to(device)
				if bit_squeezing:
					# bs = BitSqueezing(bit_depth=3)
					data = bs(data)
				if jpeg_compression:
					# jc = JPEGFilter(quality=75)
					data = jc(data).to(device)
				if average_smoothing:
					# cs = AverageSmoothing2D(kernel_size=3,channels=3)
					data = cs(data).to(device)

				return data

			def get_clean_acc(model,data,y,k=step_size_k):
				data = activate_defense(model,data,k=step_size_k,pgd_iters=pgd_iters)
				correct_classifcation = (torch.argmax(model(data), dim=1) == y).sum()
				return correct_classifcation
			clean_acc = 0

			for ibatch in range(num_batches):
				bstart = ibatch * eval_batch_size
				bend = min(bstart + eval_batch_size, num_eval_examples)
				print('batch size: {}: ({}, {})'.format(bend - bstart, bstart, bend))

				x_batch, y_batch = dset.get_eval_data(bstart, bend)
				y_batch = torch.LongTensor(y_batch).to(device)
				x_ori = torch.FloatTensor(x_batch.copy().transpose(0,3,1,2) / 255.).to(device)
				clean_acc += get_clean_acc(model, x_ori, y_batch)

				if p_norm == 'inf':
					pass
				else:
					proj_2 = l2_proj_maker(x_ori, epsilon)

				def get_label(target_type):
					_, logit = model(torch.FloatTensor(x_batch.transpose(0,3,1,2) / 255.))
					if target_type == 'random':
						label = torch.randint(low=0, high=logit.shape[1], size=label.shape).long().to(device)
					elif target_type == 'least_likely':
						label = logit.argmin(dim=1)
					elif target_type == 'most_likely':
						label = torch.argsort(logit, dim=1,descending=True)[:,1]
					elif target_type == 'median':
						label = torch.argsort(logit, dim=1,descending=True)[:,4]
					elif 'label' in target_type:
						label = torch.ones_like(y_batch) * int(target_type[5:])
					return label.detach()

				if target:
					y_batch = get_label(config["target_type"])

				if config['attack_name'] in ["SignOPTAttack","HSJAttack","GeoDAttack","OptAttack","EvolutionaryAttack",
											"SignFlipAttack","RaySAttack","BoundaryAttack"]:
					logs_dict = attacker.run(x_batch, y_batch, model, target, dset,activate_defense)

				else:
					def loss_fct(xs, es = False):
						if type(xs) is torch.Tensor:
							x_eval = (xs.permute(0,3,1,2)/ 255.).to(device)
						else:
							x_eval = (torch.FloatTensor(xs.transpose(0,3,1,2))/ 255.).to(device)

						if p_norm == 'inf':
							x_eval = torch.clamp(x_eval - x_ori, -epsilon, epsilon) + x_ori
						else:
							# proj_2 = l2_proj_maker(x_ori, epsilon)
							x_eval = proj_2(x_eval)
						x_eval = torch.clamp(x_eval, 0, 1)
						x_eval = activate_defense(model, x_eval,k= step_size_k)
						y_logit = model(x_eval.to(device))
						loss = criterion(y_logit, y_batch, target)
						if es:
							y_logit = y_logit.detach()
							correct = torch.argmax(y_logit, axis=1) == y_batch
							if target:
								return correct, loss.detach()
							else:
								return ~correct, loss.detach()
						else:
							return loss.detach()

					def early_stop_crit_fct(xs):

						if type(xs) is torch.Tensor:
							x_eval = xs.permute(0,3,1,2)/ 255.
						else:
							x_eval = torch.FloatTensor(xs.transpose(0,3,1,2))/ 255.
						x_eval = torch.clamp(x_eval, 0, 1)
						x_eval = activate_defense(model, x_eval, k= step_size_k)
						y_logit = model(x_eval.to(device))
						y_logit = y_logit.detach()
						correct = torch.argmax(y_logit, axis=1) == y_batch

						if target:
							return correct
						else:
							return ~correct

					logs_dict = attacker.run(x_batch, loss_fct, early_stop_crit_fct)

				ress = attacker.result()
				print('The Black-Box model: {}'.format(config['modeln']))
			version_dict[version] = False

			print("Batches done after {} s".format(time.time() - start_time))
			ress['Clean Accuracy'] = (ress['total_successes'] + ress['total_failures']) / num_eval_examples
			if config['dset_name'] not in res:
				res[config['dset_name']] = [ress]
			else:
				res[config['dset_name']].append(ress)

			### Only For Small experiments.
			res_fname = data_dir + '/' +  'k_{}_res.json'.format(step_size_k)
			print("Storing tabular data in {}".format(res_fname))
			with open(res_fname, 'w') as f:
				json.dump(res, f, indent=4, sort_keys=True)
'''

MIT License
Copyright (c) 2019 Abdullah Al-Dujaili
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''


Writing /content/BlackboxBench/BaselinesExperiment.py


# Hyperparameters Configuration
Change here the Hyper parameters as you like. <br>

*Uncomment the correpsonding json configuration for the attack you want to run.

In [ ]:
## NES

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'nes',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "NESAttack",
#   "attack_config": {
#     "batch_size" : 100,
#     "name": "NES",
#     "epsilon": 12.75,
#     "p": "inf",
#     "fd_eta": 2.55,
#     "lr": 2.55,
#     "q": 15,
#     "max_loss_queries": 10 # The attacker's queries budget.
#   },
#     "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed": 12345
# }

## Bandit

attack_config_json ={
  "_comment1": "===== DATASET CONFIGURATION =====",
  'name_of_file' : 'bandit',
  "dset_name": "cifar10",
  "_comment2": "===== EVAL CONFIGURATION =====",
  "num_eval_examples": 100,
  "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
  "attack_name": "BanditAttack",
  "attack_config": {
    "batch_size" :100,
    "name": "Bandit",
    "epsilon": 12.75,
    "p": "inf",
    "lr": 2.55,
    "fd_eta": 2.55,
    "prior_lr": 0.1,
    "prior_size": 20,
    "data_size": 32,
    "prior_exploration": 0.1,
    "max_loss_queries": 10000
  },
  "_comment4": "=====OUR METHOD CONFIGURATION=====",
  "defence_config":{
      "alpha" : 0.03, # The step size
      "k" : 10 #Number of iterations.
  },
  "modeln": "resnet20",
  "target": False,
  "target_type": "median",
  "seed": 123
}

## GeoD

# attack_config_json ={
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'geoda',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "GeoDAttack",
#   "attack_config": {
#     "batch_size": 1,
#     "epsilon": 12.75,
#     "p": "inf",
#     "max_queries": 10000,
#     "sub_dim": 10,
#     "tol": 0.0001,
#     "alpha": 0.0002,
#     "mu": 0.6,
#     "search_space": "sub",
#     "grad_estimator_batch_size": 40,
#     "sigma": 0
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed":123
# }

## HopSkipJump (HSJ)

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'hsja',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "HSJAttack",
#   "attack_config": {
#     "batch_size": 1,
#     "epsilon": 12.75,
#     "p": "inf",
#     "max_queries": 10000,
#     "gamma": 1.0,
#     "stepsize_search": "geometric_progression",
#     "max_num_evals": 1000,
#     "init_num_evals": 1,
#     "EOT": 0,
#     "sigma": 0
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed":123
# }

## ECO

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'parm',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "ParsimoniousAttack",
#   "attack_config": {
#     "batch_size" : 1,
#     "name": "ECO",
#     "epsilon": 12.75,
#     "p": "inf",
#     "max_loss_queries": 10000,
#     "EOT": 1,
#     "block_size": 4,
#     "block_batch_size": 64
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "sigma": 0.0,
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed": 1023
# }

## SignHunter

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'sign',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "SignAttack",
#   "attack_config": {
#     "batch_size" : 100,
#     "name": "Sign",
#     "epsilon": 12.75,
#     "p": "inf",
#     "fd_eta": 12.75,
#     "max_loss_queries": 10000
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed": 123
# }

## SignOPT

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'signopt',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "SignOPTAttack",
#   "attack_config": {
#     "batch_size": 1,
#     "epsilon": 12.75,
#     "p": "inf",
#     "alpha": 0.2,
#     "beta": 0.001,
#     "svm": False,
#     "momentum": 0,
#     "max_queries": 10000,
#     "k": 200,
#     "sigma": 0
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed":123
# }

## SimBA

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'simple',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "SimpleAttack",
#   "attack_config": {
#     "name": "SimBA",
#     "batch_size": 100,
#     "epsilon": 12.75,
#     "p": "inf",
#     "delta" : 2,
#     "max_loss_queries": 10000
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed": 123
# }


## Square

# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'square',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "SquareAttack",
#   "attack_config": {
#     "batch_size" : 100,
#     "name": "Square",
#     "epsilon": 12.75,
#     "p": "inf",
#     "p_init": 0.05,
#     "max_loss_queries": 10000
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed": 123
# }

##ZoSignSGD


# attack_config_json = {
#   "_comment1": "===== DATASET CONFIGURATION =====",
#   'name_of_file' : 'zosignsgd',
#   "dset_name": "cifar10",
#   "_comment2": "===== EVAL CONFIGURATION =====",
#   "num_eval_examples": 100,
#   "_comment3": "=====ADVERSARIAL EXAMPLES CONFIGURATION=====",
#   "attack_name": "ZOSignSGDAttack",
#   "attack_config": {
#     "batch_size" :100,
#     "name": "zosignsgd",
#     "epsilon": 12.75,
#     "p": "inf",
#     "fd_eta": 2.55,
#     "lr": 1,
#     "q": 30,
#     "max_loss_queries": 1
#   },
#   "_comment4": "=====OUR METHOD CONFIGURATION=====",
#   "defence_config":{
#       "alpha" : 0.03, # The step size
#       "k" : 10 #Number of iterations.
#   },
#   "modeln": "resnet20",
#   "target": False,
#   "target_type": "median",
#   "seed": 123
# }




In [ ]:
# Preparing the files for run.
import json
attack_json_path = f'cifar10_{attack_config_json["name_of_file"]}_linf_config.json'
# Saving the configuration file in the right place.
temp_file_path = f'/content/BlackboxBench/config-jsons/{attack_json_path}'
with open(temp_file_path, 'w') as json_file:
    json.dump(attack_config_json, json_file, indent=2)

# Running the Experiment
The results will be stored under *Results* folder.
<br>
<br>
*For choosing the baselines defences methods you want to run the experiment on, please make sure that the correspoding value in 'baselines_dict' below is 'True'* <br>
*Default - all methods*

*Reminder*: <br>
- RND : Random Noise
- SND : Gaussian Noise
- AS : Average Smoothing
- BS : Bit Squeezing
- FS : Feature Squeezing
- JPEG : Jpeg Compression
- Counter_Samples_10 : Our method with k = 10
- Counter_Samples_1 : Our method with k = 1

In [ ]:
# Choose the baselines you want to run the experiment on (True means on) - Default is all methods.
baselines_dict = {
    'FS': True,
    'RND' : True,
    'CounterSamples_10' : True,
    'Counter_Samples_1': True,
    'AS' : True,
    'BS' : True,
    'SND' : True,
    'JPEG' : True,
}

In [ ]:
# Preparing the file for run.
import json
baselines_chosed = [i for i in baselines_dict if baselines_dict[i] == True]
baselines_json_dict = {"Baselines":baselines_chosed}
baselines_json_string = json.dumps(baselines_json_dict)

In [ ]:
# Run command.
command = f"python /content/BlackboxBench/BaselinesExperiment.py /content/BlackboxBench/config-jsons/{attack_json_path} '{baselines_json_string}'"
!{command}